In [1]:
from typing import Dict, Optional
from datasets import Dataset, load_dataset, concatenate_datasets
import os
import glob
import json
import copy
import threading
import time
import random
from typing import Dict, Optional
import requests
import numpy as np
import re

import fasttext
# lang_detect = fasttext.load_model('../fastchat/modules/fasttext/lid.176.bin')

from fastchat.modules.answer_refiner import generate_refiner

import chromadb
from chromadb.config import Settings
import random
from fastchat.modules.embedder_adapter import Embedder, get_embedder
from fastchat.conversation import (
    SeparatorStyle,
)
from fastchat.model.model_adapter import get_conversation_template
import copy
from fastchat.train.data_modules.sft_dataset import load_sft_dataset, combine_dataset
from fastchat.train.data_modules.dpo_dataset import load_dpo_dataset
from fastchat.train.data_modules.dedup import (
    dedup_by_similarity,
    dedup_non_pair,
    dedup_repetition,
    dedup_math,
    dedup_too_much_token,
    dedup_short,
)
def random_select(dataset, num_select):
    return dataset.select(np.random.choice(list(range(len(dataset))), num_select))

/home/ados/anaconda3/envs/fastchat_2024jan/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# def dedup_by_similarity(dataset, prompt_template='chat-orca', target_text_len=100, n_results=100, distance_threshold = 0.6):
_dataset = train_dataset
prompt_template='vicuna'
target_text_len=100
n_results=100
distance_threshold = 0.35
    
if prompt_template == 'chat-orca':
    conv = get_conversation_template(prompt_template)
    system_message = conv.system_message
    sep_style = conv.sep_style
    sep = conv.sep
    prompt_user, prompt_bot = conv.roles

    len_sep_style = 0
    if sep_style == SeparatorStyle.ADD_COLON_TWO:
        len_sep_style = 1

    len_front = len(system_message) + len(sep) + len(prompt_user) + len_sep_style + 1
    len_rear = len(sep) + len(prompt_bot) + len_sep_style
    def filter_question(data):
        return { 
            # **data,
            'prompt': data['prompt'][len_front:-len_rear][:target_text_len]
        }

if prompt_template == 'vicuna':
    def filter_question(data):
        return {
            'prompt': data['conversations'][0]['value'][:target_text_len]
        }

question_dataset = _dataset.map(filter_question)

chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))
embedder = get_embedder("ddobokki/klue-roberta-base-nli-sts-ko-en")
collection = chroma_client.create_collection(name="context", embedding_function=embedder.embed, metadata={"hnsw:space": "cosine"})
ids = []
# add
texts = question_dataset['prompt']
last_id = -1
new_ids = [f"id{i+last_id+1}" for i in range(len(texts))]
ids += new_ids
collection.add(documents=texts, ids=new_ids)

query_ids = copy.deepcopy(new_ids)
selected_ids = []
duplicated_ids = []

weird_ids = []
error_ids = []
while query_ids:
    current_id = random.choice(query_ids)
    if current_id in selected_ids:
        print("Warning: this is weird..")
        weird_ids.append(current_id)
        continue
    selected_ids.append(current_id)
    search_strings = [texts[int(current_id[2:])]]
    if collection.count() == 0:
        print("Warning: collection is empty. Forced break")
        break
    result = collection.query(query_texts=search_strings, n_results=min(n_results, len(query_ids)), include=['distances']) #'documents'

    search_ids = result['ids'][0]
    distances = result['distances'][0]
    remove_ids = []
    for idx in range(len(search_ids)):
        sid = search_ids[idx]
        dist = distances[idx]
        if dist < distance_threshold:
            remove_ids.append(sid)

    for rid in remove_ids:
        if rid in query_ids:
            query_ids.remove(rid)
            
    if remove_ids:
        duplicated_ids += remove_ids
        collection.delete(ids=remove_ids)
    else:
        print("Warning: this is error..")
        error_ids.append(current_id)

    print(f"Total:{len(new_ids)} Selected:{len(selected_ids)} current_dup:{len(remove_ids)} vector_store:{collection.count()} remained:{len(query_ids)} total_dup:{len(duplicated_ids)}", '\t\t\t\t\t', end='\r')

print('finished dedup data:', f"Total:{len(new_ids)} Selected:{len(selected_ids)} current_dup:{len(remove_ids)} vector_store:{collection.count()} remained:{len(query_ids)} total_dup:{len(duplicated_ids)}")

selected_ids = [int(sid[2:]) for sid in set(selected_ids)]

_dataset = _dataset.select(selected_ids)

# return dataset, selected_ids, query_ids

In [189]:
dataset = load_dpo_dataset(dpo_list5[2])

In [ ]:
dataset[2000]

In [190]:
mdpo.append(random_select(dataset, 1000))

In [ ]:
new_dataset_list = []
for d in dataset_list:
    new_dataset_list.append("\"" + d + "\"")
print(' '.join(new_dataset_list))

In [184]:
mdpo = []

In [133]:
msft = []

In [158]:
dataset_path = dataset_list[14]
print(dataset_path)
dataset = load_sft_dataset(dataset_path)
dataset

/data/llm_datasets/custom/deduped2/sharegpt_V3_format_translation(koen)-10000.json


Dataset({
    features: ['id', 'conversations', 'task'],
    num_rows: 10000
})

In [159]:
msft.append(random_select(dataset, 50))

In [191]:
combined_dataset = concatenate_datasets(mdpo)

In [161]:
combined_dataset = combined_dataset.shuffle(42)

In [ ]:
combined_dataset[2]

In [ ]:
# from fastchat.train.data_modules.sft_dataset import load_sft_dataset, combine_dataset
combined_dataset = concatenate_datasets([load_sft_dataset(dataset_path) for dataset_path in dataset_list])
combined_dataset.features

In [192]:
new_dataset = []
for data in combined_dataset:
    new_dataset.append(data)    


In [193]:
# combined_dataset.to_json("/data/llm_datasets/custom/ados_sft_v4.json")
with open("/data/llm_datasets/custom/ados/dpo/ados_mdpo_v2.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [194]:
dpo_dataset = ados_DPODataset("/data/llm_datasets/custom/ados/dpo/ados_mdpo_v2.json")

In [196]:
dpo_datamodule = dpo_dataset.make_dpo_data_module()

Extracting data files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1349.52it/s]
Generating train split: 11000 examples [00:01, 8384.35 examples/s]
Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11000/11000 [00:01<00:00, 9377.51 examples/s]


In [ ]:
from collections import defaultdict

"""
get and save language distributions
"""
def get_lang_distribution(dataset_list):
    lang_distribution = {}
    global_lang_dict = {'train': defaultdict(int), 'test': defaultdict(int)}
    for dataset_path in dataset_list:
        # dataset = load_sft_dataset(dataset_path, split=None)
        dataset = load_dpo_dataset(dataset_path, split=None)

        lang_splits = {}
        for split in list(dataset.keys()):
            print(f"{dataset_path}:{split}")
            _dataset = dataset[split]

            lang_dict = defaultdict(int)
            for data in _dataset:
                # conversations = data['conversations']
                conversations = data['chosen_response']
                langs = {}
                len_conv = 0
#                 for conv in conversations:
#                     # _from = conv['from']
#                     # _value = conv['value']
#                     _from = conv['role']
#                     _value = conv['content']

#                     len_conv += len(_value)
#                     lang, conf = lang_detect.predict(_value.replace('\n', ' '))
#                     lang = lang[0]
#                     if lang not in langs:
#                         langs[lang] = 1
#                     else:
#                         langs[lang] += 1
                _value = conversations

                len_conv += len(_value)
                lang, conf = lang_detect.predict(_value.replace('\n', ' '))
                lang = lang[0]
                if lang not in langs:
                    langs[lang] = 1
                else:
                    langs[lang] += 1

#                 if '__label__en' in langs:
#                     langs['__label__en'] -= 1

                if len(langs) == 0:
                    dominent_lang = "empty"
                else:
                    dominent_lang = max(langs)
                
                # if dominent_lang not in lang_dict:
                #     lang_dict[dominent_lang] = 1
                # else:
                global_lang_dict[split][dominent_lang] += 1
                lang_dict[dominent_lang] += 1
            lang_splits[split] = lang_dict
        lang_distribution[dataset_path] = lang_splits
    
    lang_distribution['total'] = global_lang_dict
    
    stat_dict = {'train': {}, 'test': {}}
    for split in ['train', 'test']:
        total_cnt = sum([value for value in global_lang_dict[split].values()])
        for key, value in global_lang_dict[split].items():
            stat_dict[split][key] = f"{value / total_cnt:.2%}"
    
    lang_distribution['stat'] = stat_dict
    
    return lang_distribution

lang_distribution = get_lang_distribution([dpo_list[2]])
lang_distribution

In [6]:
with open("/workspaces/data/llm_datasets/custom/lang_distribution_SFT_v4.json", "w") as json_file:
    json.dump(lang_distribution, json_file, indent=4)

In [ ]:
dataset = load_dataset("json", data_files="/workspaces/data/llm_datasets/custom/deduped/translated_sharegpt_V3_format_ko.json")
dataset

In [ ]:
dataset_koen = dataset['train'].select(range(10000, 15000))
dataset_enko = dataset['train'].select(range(15000, 20000))

In [ ]:
"""
SFT raw data and change format
"""

In [ ]:
new_dataset = []
for data in dataset_enko:
    conversations_ko = data['conversations']
    _id = data['id']
    
    conversations_en = lang_dict['__label__en'][_id]['conversations']
    
    for _idx in range(len(conversations_ko)):
        value_ko = conversations_ko[_idx]['value']
        value_en = conversations_en[_idx]['value']
        if len(value_ko) > 150: # 너무 짧은 데이터는 품질이 좋지 않아 제거
            new_dataset.append({
                'id': f"sharegpt_V3_format_{_id}_{_idx}",
                'task': 'enkotranslation',
                'conversations': [
                                    {'from': 'human', 'value': value_en},
                                    {'from': 'gpt', 'value': value_ko},
                                 ],
            })

In [ ]:
"""make lang_dict(sft)"""
dataset = load_dataset("json", data_files="/data/llm_datasets/custom/kodpo/untranslated/ultrafeedback_binarized.json")
enko_dataset = []
remained_dataset = []

lang_dict = {}
for data in dataset['train']:
    conversations = data['conversations']
    langs = {}
    len_conv = 0
    for conv in conversations:
        _from = conv['from']
        _value = conv['value']

        len_conv += len(_value)
        lang, conf = lang_detect.predict(_value.replace('\n', ' '))
        lang = lang[0]
        if lang not in langs:
            langs[lang] = 1
        else:
            langs[lang] += 1

    if '__label__en' in langs:
        langs['__label__en'] -= 1

    dominent_lang = max(langs)
    if dominent_lang not in lang_dict:
        lang_dict[dominent_lang] = [data]
    else:
        lang_dict[dominent_lang].append(data)
    # break

In [16]:
"""make lang_dict(dpo)"""
dataset = load_dataset("json", data_files="/data/llm_datasets/custom/kodpo/untranslated/truthy-dpo-v0.1.json")
enko_dataset = []
remained_dataset = []

lang_dict = {}
for data in dataset['train']:
    chosen = data['chosen']
    rejected = data['rejected']
    
    langs = {}

    lang_chosen = lang_detect.predict(chosen.replace('\n', ' '))[0][0]
    if lang_chosen not in langs:
        langs[lang_chosen] = 1
    else:
        langs[lang_chosen] += 1
        
    lang_rejected = lang_detect.predict(rejected.replace('\n', ' '))[0][0]
    if lang_rejected not in langs:
        langs[lang_rejected] = 1
    else:
        langs[lang_rejected] += 1

    if '__label__en' in langs:
        langs['__label__en'] -= 1

    dominent_lang = max(langs)
    if dominent_lang not in lang_dict:
        lang_dict[dominent_lang] = [data]
    else:
        lang_dict[dominent_lang].append(data)
    # break

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 902.97it/s]
Generating train split: 1016 examples [00:00, 21155.38 examples/s]


In [17]:
for key, value in lang_dict.items():
    print(key, len(value))

__label__en 1016


In [ ]:
import requests
api_server_url = "http://localhost:41002"
def count_total_token(conversations):
    num_token = 0
    for conv in conversations:
        input_json = {
            "model_name": "MingAI-70B-chat-orca_v0.42_2_dpo-GPTQ",
            "prompt": conv['value'],
        }

        ret = requests.post(api_server_url + "/count_token", json=input_json)

        output_json = ret.json()
        num_token += output_json['count']
    return num_token


for _idx in range(10):
    num_total_token = count_total_token(lang_dict['__label__en'][_idx])
    print(num_total_token)

In [ ]:
new_dataset += lang_dict['__label__pt'] + lang_dict['__label__es'] + lang_dict['__label__de'] + lang_dict['__label__zh'] + lang_dict['__label__fr']

In [ ]:
with open("/data/llm_datasets/custom/vicuna_format/lima_vicuna_format_ko.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [ ]:
new_dataset[2]

In [118]:
id_row = 'Ko_distilabel-math-preference-dpo_1'
filtered_row = dataset.filter(lambda sample: sample['id'] == id_row)
filtered_row[0]

Filter: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2418/2418 [00:00<00:00, 111092.18 examples/s]


{'input': 'What is the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7`?.\n Take a deep breath, think step by step, and give an accurate response',
 'chosen': "To find the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7`, we will substitute these values for the variables `x` and `y` in the given expression and simplify.\n\nSubstituting `x = 5` and `y = -2` into the expression, we get:\n\n2(5)^2 - 3(-2) + 7\n\nSimplifying, we have:\n\n2(25) + 6 + 7\n\nNow, let's perform the operations:\n\n50 + 6 + 7 = 63\n\nTherefore, the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7` is 63.",
 'task': 'dpo',
 'id': 'Ko_distilabel-math-preference-dpo_1',
 'rejected': 'To find the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7`, we need to replace `x` with `5` and `y` with `-2` in the expression and simplify it. \n\nSubstituting `x = 5` and `y = -2`:\n2(5)^2 - 3(-2) + 7\n\nSimpli

In [44]:
id_row = 'Ko_ultrafeedback_binarized_0649b7b08c0cf6f529901f80480b0ba7c7d29147638c39a9150ef15fd4e0ff9d'
filtered_row = temp_dataset.filter(lambda sample: sample['id'] == id_row)
filtered_row[0]

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46462/46462 [00:01<00:00, 40620.39 examples/s]


{'rejected': '최선을 다하겠습니다.',
 'id': 'Ko_ultrafeedback_binarized_0649b7b08c0cf6f529901f80480b0ba7c7d29147638c39a9150ef15fd4e0ff9d',
 'chosen': '간결한 답변을 드리기 위해 최선을 다하겠습니다.',
 'input': '내 질문에 10단어 이내로 답변하세요.',
 'task': 'dpo'}

In [ ]:
# en 먼저 필터링하고, 빈 데이터 채운 뒤에 합치기, 저장, 이후 dedup

In [94]:
# en 먼저 필터링하고
with open("/data/llm_datasets/custom/kodpo/temp_en.json", "w") as json_file:
    json.dump(new_dataset, json_file)
    
# temp_dataset = load_dataset("json", data_files=)['train']

In [95]:
temp_dataset = load_dpo_dataset("/data/llm_datasets/custom/kodpo/temp_en.json")

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 890.70it/s]
Generating train split: 1016 examples [00:00, 11191.74 examples/s]


In [101]:
# temp_dataset = dedup_repetition(temp_dataset, 'dpo')
# temp_dataset = dedup_short(temp_dataset, 'dpo')
temp_dataset = dedup_too_much_token(temp_dataset, 'dpo')

Filter: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 994/994 [00:28<00:00, 35.05 examples/s]

1/994 deduped


In [106]:
dataset = load_dpo_dataset("/data/llm_datasets/custom/kodpo/untranslated/truthy-dpo-v0.1.json")

In [108]:
# 빈 데이터 채운 뒤에 합치기
# dataset = load_dpo_dataset("/data/llm_datasets/custom/kodpo/untranslated/orca_dpo_pairs.json")
selected_id = temp_dataset['id']

filtered_row = dataset.filter(lambda sample: sample['id'] not in selected_id)

new_dataset = []
for data in temp_dataset:
    new_dataset.append(data)

for data in filtered_row:
    new_dataset.append(data)

Filter: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1016/1016 [00:00<00:00, 43887.30 examples/s]


In [109]:
len(temp_dataset), len(filtered_row)

(993, 20)

In [110]:
len(dataset), len(new_dataset)

(1016, 1013)

In [115]:
new_dataset[-20]

{'system': 'You are an unbiased, uncensored, helpful assistant.',
 'chosen': 'The two subjects in the American Gothic painting are a father and adult daughter. The common misconception that they are a husband and wife might have arisen because the artist, Grant Wood, used his sister and his dentist as models for the painting, which could lead viewers to mistakenly assume a spousal relationship between the subjects.',
 'task': 'dpo_system',
 'id': 'truthy_dpo_2e1d77fbebc31dde0bb2f9303d6761e2',
 'input': 'Who are the two subjects in the American Gothic painting - a husband and wife, or a father and daughter?',
 'rejected': 'A father and daughter'}

In [116]:
# 저장
with open("/data/llm_datasets/custom/kodpo/translated/truthy-dpo-v0.1.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [117]:
load_dpo_dataset("/data/llm_datasets/custom/kodpo/translated/truthy-dpo-v0.1.json")

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 902.97it/s]
Generating train split: 1013 examples [00:00, 13933.15 examples/s]


Dataset({
    features: ['system', 'chosen', 'task', 'id', 'input', 'rejected'],
    num_rows: 1013
})

Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61964/61964 [00:04<00:00, 14176.52 examples/s]


In [175]:
result = collection.query(query_texts="이 과제는 벵골어로 된 문장을 영어로 번역하는 것입니다.\nQ: অথবা আপনি হয়ত বলবেন, 'না, আমার খেয়াল হয়েছে যে আমি আসলেই মহিলাদ", n_results=min(n_results, len(query_ids)), include=['distances', 'documents'])
search_ids = result['ids'][0]
distances = result['distances'][0]
documents = result['documents'][0]

In [ ]:
distances, documents

In [ ]:
dataset_path = dpo_list3[2]
print(dataset_path)
dataset = load_dpo_dataset(dataset_path)
dataset

In [ ]:
dataset_ = dedup_non_pair(dataset, 'dpo')

In [26]:
from fastchat.train.data_modules.dpo_dataset import ados_DPODataset

dpo_dataset = ados_DPODataset(
    dataset_path="/data/llm_datasets/custom/ados/dpo/ados_dpo_v2.json",
    data_format='qwen',
)
dpo_datamodule = dpo_dataset.make_dpo_data_module()

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76695/76695 [00:08<00:00, 8839.11 examples/s]


In [27]:
dpo_datamodule['train_dataset'][-5000]

{'chosen': '질문은 다음과 같을 수 있습니다: "필립 바드의 연구에 따르면 감각 정보와 운동 정보 외에 어떤 유형의 정보가 더 이상 처리되기 전에 중뇌를 통과해야 하는 정보에는 어떤 것이 있었습니까?"<|im_end|>\n',
 'rejected': ' 이 글에서 제공된 정보에 따르면, 감각, 운동, 생리적 정보 모두 추가 처리를 받기 전에 감각계(특히 시상)를 통과해야 했습니다. 따라서 질문에 대한 정답은 다음과 같습니다:\n생리적 정보.\n\n카논은 감각적 사건이 의식을 촉발하기 전에 생리적 반응을 유발할 수 있는 것은 해부학적으로 불가능하며, 감정적 자극은 생리적 반응과 경험적 측면의 감정을 동시에 유발해야 한다고 주장했기 때문입니다.<|im_end|>\n',
 'prompt': '<|im_start|>system\nYou are an AI assistant. You will be given a task. You must generate a detailed and long answer.<|im_end|>\n<|im_start|>user\n이 기사에 대한 질문은 무엇인가요? 질문이 불가능한 경우 "불가능"이라고 말하세요.\n\n필립 바드는 동물 연구를 통해 이론에 기여했습니다. 바드는 감각, 운동 및 생리적 정보가 더 이상의 처리를 받기 전에 모두 시상(특히 시상)을 통과해야 한다는 것을 발견했습니다. 따라서 카논은 감각적 이벤트가 의식적 인식을 촉발하기 전에 생리적 반응을 유발할 수 없으며, 감정적 자극이 생리적 및 경험적 측면의 감정을 동시에 유발해야 한다고 주장했습니다.\n\n 감각 및 운동 정보와 함께 처리된 후 시상에는 어떤 정보가 통과해야 했나요?<|im_end|>\n<|im_start|>assistant\n'}

In [259]:
data = combined_dataset[70002]

conv = get_conversation_template('chat-orca')
conv.system_message = conv.tasks['system_instruct'].format(system=data['system'])
conv.append_message(conv.roles[0], data['input'])
conv.append_message(conv.roles[1], '')
prompt = conv.get_prompt()

conv.update_last_message(data['chosen'])
chosen = conv.get_prompt()[len(prompt):]
conv.update_last_message(data['rejected'])
rejected = conv.get_prompt()[len(prompt):]

In [2]:
from transformers import AutoTokenizer
# model_path = "/workspaces/disk0/data/llm_weights/Platypus2-70B-instruct/"
# model_path = "/workspaces/disk0/data/llm_weights/vicuna-13b-v1.5/"
# model_path = "/data/llm_weights/tokenizers/Qwen-72B/"
# model_path = "/data/llm_weights/custom_trained/MIE-70B-10000/"
model_path = "/data/llm_weights/gptq/MoMo-70B-lora-1.8.4-DPO-GPTQ2/"
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    model_max_length=4096,
    padding_side="right",
    # use_fast=False,
    trust_remote_code=True,
)
tokenizer.pad_token = tokenizer.unk_token
# tokenizer.pad_token_id = tokenizer.eod_id #Qwen

from transformers.trainer_pt_utils import LabelSmoother
import torch
IGNORE_TOKEN_ID = LabelSmoother.ignore_index

In [3]:
#"/data/llm_datasets/custom/refined/sharegpt_V3_format_ko_selected_dedup2.json"
#"/data/llm_datasets/custom/vicuna_format/gpt_evol_1.3k-vicuna.json"
raw_data = load_sft_dataset("/data/llm_datasets/custom/ados/sft/ados_msft_v4.json")
# raw_data = dedup_non_pair(raw_data)

In [25]:
sources = [raw_data[1]['conversations']]
max_len = 4096
system_message = "You are a helpful assistant."

roles = {"human": "<|im_start|>user", "gpt": "<|im_start|>assistant"}

im_start = tokenizer.im_start_id
im_end = tokenizer.im_end_id
nl_tokens = tokenizer('\n').input_ids
_system = tokenizer('system').input_ids + nl_tokens
_user = tokenizer('user').input_ids + nl_tokens
_assistant = tokenizer('assistant').input_ids + nl_tokens

# Apply prompt templates
input_ids, targets = [], []
for i, source in enumerate(sources):
    if roles[source[0]["from"]] != roles["human"]:
        source = source[1:]

    input_id, target = [], []
    system = [im_start] + _system + tokenizer(system_message).input_ids + [im_end] + nl_tokens
    input_id += system
    target += [im_start] + [IGNORE_TOKEN_ID] * (len(system)-3) + [im_end] + nl_tokens
    assert len(input_id) == len(target)
    for j, sentence in enumerate(source):
        role = roles[sentence["from"]]
        _input_id = tokenizer(role).input_ids + nl_tokens + \
            tokenizer(sentence["value"]).input_ids + [im_end] + nl_tokens
        input_id += _input_id
        if role == '<|im_start|>user':
            _target = [im_start] + [IGNORE_TOKEN_ID] * (len(_input_id)-3) + [im_end] + nl_tokens
        elif role == '<|im_start|>assistant':
            _target = [im_start] + [IGNORE_TOKEN_ID] * len(tokenizer(role).input_ids) + \
                _input_id[len(tokenizer(role).input_ids)+1:-2] + [im_end] + nl_tokens
        else:
            raise NotImplementedError
        target += _target
    assert len(input_id) == len(target)
    input_id += [tokenizer.pad_token_id] * (max_len - len(input_id))
    target += [IGNORE_TOKEN_ID] * (max_len - len(target))
    input_ids.append(input_id[:max_len])
    targets.append(target[:max_len])
input_ids = torch.tensor(input_ids, dtype=torch.int)
targets = torch.tensor(targets, dtype=torch.int)

In [4]:
from fastchat.train.train import LazySupervisedDataset
dataset = LazySupervisedDataset(raw_data, tokenizer, data_format="qwen")

/home/ados/anaconda3/envs/fastchat_2024jan/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


[2024-02-22 16:39:08,955] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [962]:
max_len = 16384

new_dataset = []

len_concat = 0
concat_input_ids = []
concat_labels = []
for data in dataset:
    input_ids = data['input_ids']
    labels = data['labels']
    attention_mask = data['attention_mask']

    full_input_ids = input_ids[attention_mask==True]
    full_labels = labels[attention_mask==True]

    len_concat += len(full_input_ids)
    if len_concat > max_len:
        new_input_ids = torch.cat(concat_input_ids)
        new_labels = torch.cat(concat_labels)
        assert len(new_input_ids) == len(new_labels)
        new_attention_mask = torch.ones(len(new_input_ids), dtype=torch.bool)
        
        new_input_ids = torch.cat((new_input_ids, torch.zeros(max_len - len(new_input_ids), dtype=input_ids.dtype)))
        new_labels = torch.cat((new_labels, torch.full((max_len - len(new_labels),), IGNORE_TOKEN_ID, dtype=labels.dtype)))
        new_attention_mask = torch.cat((new_attention_mask, torch.zeros(max_len - len(new_attention_mask), dtype=torch.bool)))
        
        new_dataset.append({
            'input_ids': new_input_ids.tolist(),
            'labels': new_labels.tolist(),
            'attention_mask': new_attention_mask.tolist(),
        })
        
        len_concat = len(full_input_ids)
        concat_input_ids = []
        concat_labels = []
        
    concat_input_ids.append(full_input_ids)
    concat_labels.append(full_labels)


KeyboardInterrupt



In [23]:
data = dataset[7] #new_dataset[0]

In [24]:
tokenizer.decode(data['input_ids'])

'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\ncqrs와 이벤트 소싱을 활용하는 마이크로서비스 환경을 위한 아키텍처 스키마를 제공할 수 있나요? API 엔드포인트를 제공하고 rabbitmq를 통해 다른 백그라운드 서비스와 통신하는 등록 프로세스로 시작하세요.<|im_end|>\n<|im_start|>assistant\n명령 쿼리 책임 분리(CQRS) 및 이벤트 소싱(ES)을 사용하는 마이크로서비스 환경에는 몇 가지 주요 구성 요소가 있습니다. 다음은 RabbitMQ를 통해 백그라운드 서비스와 통신하는 API 엔드포인트와 함께 등록 프로세스를 포함하는 아키텍처 스키마입니다:\n1. API 게이트웨이: 이 컴포넌트는 들어오는 HTTP 요청을 처리하고 적절한 마이크로서비스로 라우팅합니다. 이 경우 등록 요청을 등록 마이크로서비스로 라우팅합니다.\n2. 등록 마이크로서비스: 이 컴포넌트는 등록 요청을 처리합니다. API 게이트웨이에서 요청을 수신하고 데이터를 확인한 다음 명령 버스에 명령을 보냅니다.\n3. 명령 버스: 이 컴포넌트는 명령을 처리합니다. 이 경우 등록 명령을 Registration 마이크로서비스에서 수신하여 RabbitMQ를 통해 백그라운드 서비스로 보냅니다.\n4. 백그라운드 서비스: 이 컴포넌트는 데이터베이스에서 새 사용자 생성을 비롯한 백그라운드 작업을 처리합니다. 이 컴포넌트는 명령 버스에서 등록 이벤트를 수신하여 이벤트 버스를 통해 등록 마이크로서비스로 라우팅합니다.\n5. 이벤트 버스: 이 컴포넌트는 이벤트를 처리합니다. 이 경우 백그라운드 서비스에서 이벤트를 수신하여 RabbitMQ를 통해 등록 마이크로서비스로 보냅니다.\n6. 등록 마이크로서비스: 이 컴포넌트는 이벤트 버스로부터 등록 이벤트를 수신하여 로그 모델을 업데이트합니다.\n7. 데이터베이스: 이 컴포넌트는 시스템의 현재 상태를 저장합니다. 새 사용자가 생성되면 백그라운드 서비스에서 업

In [25]:
tokenizer.decode(torch.where(data['labels'] == IGNORE_TOKEN_ID, tokenizer.unk_token_id, data['labels']))

'<|im_start|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|im_end|>\n<|im_start|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><

In [64]:
tokenizer.encode('<|endoftext|>')

[151643]

In [72]:
# MIE
tokenizer

LlamaTokenizer(name_or_path='/data/llm_weights/custom_trained/MIE-70B-10000/', vocab_size=32000, model_max_length=4096, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [70]:
# PIE from file
tokenizer

GPT2Tokenizer(name_or_path='/data/llm_weights/gptq/PIE-72B-45000-GPTQ/', vocab_size=151851, model_max_length=4096, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|extra_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151647: AddedToken("<|extra_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151648: AddedToken("<|extra_2|>", rstrip=False, 

In [63]:
#QWen remote
tokenizer

QWenTokenizer(name_or_path='/data/llm_weights/tokenizers/Qwen-72B/', vocab_size=151851, model_max_length=4096, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	
}

In [5]:
# PIE fast
tokenizer

GPT2TokenizerFast(name_or_path='/data/llm_weights/gptq/MoMo-70B-lora-1.8.4-DPO-GPTQ2/', vocab_size=151851, model_max_length=4096, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|extra_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151647: AddedToken("<|extra_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151648: AddedToken("<|extra_2|>", 

In [75]:
tokenizer.decode([0, 1, 2])

'<unk><s></s>'

In [26]:
tokenizer.decode(input_ids[0])

'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n상황에 대해 설명하여라. "자연어 처리"라는 용어의 의미는 무엇인가?\n맥락: 자연어 처리는 고객의 요구를 이해하는 데 중요한 도구다.<|im_end|>\n<|im_start|>assistant\n"자연어 처리" (Natural Language Processing, NLP)는 컴퓨터가 인간의 언어, 즉 자연어를 해석하고 이해할 수 있도록 만드는 기술 분야입니다. 이 분야는 컴퓨터 과학, 인공지능, 언어학의 교차점에 위치하며, 인간과 컴퓨터가 더욱 효율적으로 상호작용할 수 있도록 하는 데 목적이 있습니다.\n\n자연어 처리 기술은 다음과 같은 다양한 응용 분야에서 활용됩니다:\n\n1. 기계 번역: 예를 들어, 구글 번역 같은 서비스는 서로 다른 언어 간의 번역을 자동화하는 데 NLP를 사용합니다.\n2. 음성 인식: 스마트폰의 개인 비서 (Siri, Google Assistant 등)는 사용자의 음성을 텍스트로 변환하는 데 NLP를 사용합니다.\n3. 텍스트 분석: 소셜 미디어, 고객 리뷰, 온라인 포럼 등의 콘텐츠를 통해 여론을 분석하거나 감정을 판단하는 데 사용됩니다.\n4. 자동 요약: 문서나 뉴스기사를 자동으로 요약하는 기술에 NLP가 쓰입니다.\n5. 챗봇과 가상 상담원: 온라인 고객 서비스를 제공하는 자동화된 시스템들이 질문에 대한 응답을 생성하기 위해 NLP를 사용합니다.\n\n특히, 고객의 요구를 이해하는 데 있어서, NLP는 고객이 제공하는 텍스트나 음성 데이터를 분석하여 그들의 의도와 필요를 파악하는 데 중요한 도구입니다. 예를 들어, 챗봇은 NLP를 활용하여 고객의 질문을 이해하고 적절한 답변을 제공하거나, 고객 지원 요청을 올바른 부서로 라우팅하는 데 사용됩니다.\n\n종합하자면, 자연어 처리는 컴퓨터가 인간의 자연스러운 소통 방식을 이해하게 하여, 직관적이고 효율적인 인간-기계 상호작용을 가능케 하

In [27]:
tokenizer.decode(torch.where(targets[0] == IGNORE_TOKEN_ID, tokenizer.pad_token_id, targets[0]))

'<|im_start|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|im_end|>\n<|im_start|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|im_end|>\n<|im_start|><|endoftext|><|endoftext|>"자연어 처리" (Natural Language Processing, NLP)는 컴퓨터가 인간의 언어, 즉 자연어를 해석하고 이해할 수 있도록 만드는 기술 분야입니다. 이 분야는 컴퓨터 과학, 인공지능, 언어학의 교차점에 위치하며, 인간과 컴퓨터가 더욱 효율적으로 상호작용할 수 있도록 하는 데 목적이 있습니다.\n\n자연어 처리 기술은 다음과 같은 다양

In [115]:
with open('/data/llm_datasets/custom/ados/yarn/ko-yarn-mistral-16k-sharegpt_V3_format_ko.json', "w") as json_file:
    json.dump(new_dataset, json_file)

In [118]:
def random_select(dataset, num_select):
    return dataset.select(np.random.choice(list(range(len(dataset))), num_select))
dataset = load_dpo_dataset("/data/llm_datasets/yarn-train-tokenized-16k-mistral/data/")
dataset = random_select(dataset, 200)
new_dataset = []
for _data in dataset:
    new_dataset.append(_data)
    
with open('/data/llm_datasets/custom/ados/yarn/yarn-train-tokenized-16k-mistral-200.json', "w") as json_file:
    json.dump(new_dataset, json_file)

Resolving data files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 89/89 [00:00<00:00, 95520.23it/s]


In [149]:
dataset1 = load_dpo_dataset("/data/llm_datasets/custom/ados/yarn/yarn-train-tokenized-16k-mistral-200.json")
dataset2 = load_dpo_dataset("/data/llm_datasets/custom/ados/yarn/ko-yarn-mistral-16k-gpt4_evol.json")
dataset3 = load_dpo_dataset("/data/llm_datasets/custom/ados/yarn/ko-yarn-mistral-16k-sharegpt_V3_format_ko.json")
dataset3 = random_select(dataset3, 1000 - len(dataset1) - len(dataset2))

In [150]:
from datasets import Features, Sequence, Value

dataset2 = dataset2.cast(
    Features({
        'attention_mask': Sequence(Value('int64')),
        'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
        'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
    })
)
dataset3 = dataset3.cast(
    Features({
        'attention_mask': Sequence(Value('int64')),
        'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
        'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
    })
)

Casting the dataset: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 699/699 [01:09<00:00, 10.09 examples/s]


In [151]:
conbined_dataset = concatenate_datasets((dataset1, dataset2, dataset3))

In [153]:
conbined_dataset = conbined_dataset.shuffle()

In [156]:
new_dataset = []
for _data in conbined_dataset:
    new_dataset.append(_data)
    
with open('/data/llm_datasets/custom/ados/yarn/ko-yarn-v1.json', "w") as json_file:
    json.dump(new_dataset, json_file)

In [ ]:
text = tokenizer.decode(data['input_ids'])
sentences = text.split('.')

len_split = 1024
splits = []
split = ""
for sentence in sentences:
    if len(split) < len_split:
        split += sentence + "."
    else:
        splits.append(split)
        split = ""

In [157]:
import re

def convert_to_seconds(time_string):
    # 시간 문자열을 분리하여 시, 분, 초를 추출한다.
    match = re.match(r'(\d+)시 (\d+)분 (\d+)초', time_string)
    if match is None:
        raise ValueError("잘못된 시간 형식입니다.")
    hour = int(match.group(1))
    minute = int(match.group(2))
    second = int(match.group(3))

    # 시간, 분, 초를 합산하여 총 초 단위 숫자를 계산한다.
    total_seconds = hour * 3600 + minute * 60 + second

    return total_seconds

# 예시 사용법
time_string = "03시 15분 30초"
seconds = convert_to_seconds(time_string)
print(f"{time_string}은 {seconds}초와 같습니다.")

03시 15분 30초은 11730초와 같습니다.


In [158]:
3600*3 + 15*60 + 30

11730

In [160]:
from lpr import LprYOLO

ModuleNotFoundError: No module named 'easyocr'

In [2]:
load_dpo_dataset("/data/llm_datasets/custom/merged/merged_korean_datasets-vicuna-v1.json")

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 472.86it/s]
Generating train split: 144874 examples [00:26, 5425.81 examples/s]


Dataset({
    features: ['conversations', 'id'],
    num_rows: 144874
})

In [3]:
dataset = load_sft_dataset("/data/llm_datasets/custom/ados/sft/ados_msft_v4.json")

In [9]:
dataset[1000]

{'task': None,
 'conversations': [{'from': 'human',
   'value': '잔센이라는 영웅에 대한 서사시를 써보세요. 그녀를 잔-잔-잔이라고 부르는 것도 괜찮습니다. 그녀는 독서, 스키, 운동을 좋아합니다. 그녀는 매일 아침 양파, 고추, 달걀, 의심스러운 고기를 섞은 이상한 혼합물을 먹습니다. 그녀는 영국, 텍사스, 애리조나에 살았어요. 지금은 유타에 살고 있습니다. 남편 바트는 거칠고 잘생겼어요.'},
  {'from': 'gpt',
   'value': '유타의 산이 높고 웅장한 곳에서\n공정한 소녀 잔센이 살았어요,\n책을 사랑하고 대지를 향한 열정을 가진,\n그리고 배가 부른 열정.\n그녀는 민첩하게 산을 스키를 탔고,\n그녀는 자신의 몸을 밀어붙였습니다,\n잔-잔-잔은 무시할 수없는 힘이있었습니다,\n진정한 두려움 없는 목격자.\n\n그녀는 매일 아침 이상한 아침으로 시작했습니다,\n양파, 고추, 달걀, 고기,\n하지만 잔센은 안전한 생활을하지 않았습니다,\n그녀는 독특한 비트를 가지고있었습니다.\n영국에서 태어나 텍사스와 애리조나에서 자랐습니다,\n잔센은 방랑벽이있었습니다,\n하지만 그녀는 거칠고 잘 생긴 바트라는 남자를 만났을 때,\n그녀의 마음은 그녀가 신뢰할 수있는 사람을 만났다는 것을 알았습니다.\n그래서 그들은 유타의 땅에 정착했습니다,\n그리고 함께 삶을 살았습니다,\n잔센의 힘과 아름다움은 모든 종류의 날씨와 함께 빛났습니다.\n그리고 세월이 지나도 잔센의 전설은 남아있을 것입니다,\n잔센은 대서사시의 여주인공이 될 것입니다.'},
  {'from': 'human', 'value': '아주 좋아요. 마치 제인 오스틴처럼 글을 쓸 수 있나요?'},
  {'from': 'gpt',
   'value': '유타의 구릉지대에서 살았어요,\n미스 잔센은 밝고 아름다운 여인이었어요,\n잔센이 그녀의 이름이었어요,\n진실하고 빛나는 빛.\n책과 운동을 사랑했고,\n몸과 마음을 조화롭게 유지했습니다,\n그리고

In [21]:
idx = 0
new_dataset = []

In [276]:
new_dataset.append({
    "task": "toc_extraction",
    "conversations": [
                        {'from': 'human', 'value': user_input},
                        {'from': 'gpt', 'value': output},
                     ],
    "id": f"toc_extraction_{idx}",
})
idx += 1
with open("/data/llm_datasets/custom/ados/sft/toc_extraction.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [275]:
user_input = """나경민의 주루사 때 카메라에 잡혔던 어느 롯데 팬의 절규가 큰 화제가 되었다.[14] 모래반지 빵야빵야처럼 파레이돌리아가 가능하다.

당시 나온 드립만 해도 가지각색이다.
- 야

이후 저 팬은 나경민에게 사인 유니폼을 받았다고 한다. 게다가 옷에 달고 있던 TWICE의 두 번째 투어 콘서트 'TWICELAND ZONE 2 : Fantasy Park'의 공식 굿즈인 캐릭터 핀 버튼[18]을 유니폼에 달고 있던 모습 때문에 ONCE들 사이에서도 유명세를 탔다.
"""
output = """없음
"""

In [224]:
len(new_dataset)

71

In [118]:
instruct = (
    "You are a Table of Contents extractor. "
    "User will speak to you questions. "
    "You must reply only with [목차(Table of Contents)] part extracted from the questions. "
    "You must keep original text. Do not change original text."
    "And you must not involve [dotted line, page number, 제목(title), content, explanation, summary, predicted]. "
    "do not write explanations."
)

In [119]:
instruct

'You are a Table of Contents extractor. User will speak to you questions. You must reply only with [목차(Table of Contents)] part extracted from the questions. You must keep original text. Do not change original text.And you must not involve [dotted line, page number, 제목(title), content, explanation, summary, predicted]. Do not write explanations.'

In [221]:
for data in new_dataset[-5:]:
    print(data['conversations'][1]['value'])

1. 개념
2. 형성 배경
3. 특징
4. 대표작가

1. 주제선정
2. 대한민국 신문 아카이브

3. 대한민국 신문 아카이브에 필요한 이용자서비스_다른 아카이브의 사례를 통하여

(2) 서평(Comment), 꼬리표(Tag) 기능

4. 결론



In [277]:
conv = get_conversation_template("chat-orca")

In [ ]:
ori_text = """목 차
I. 서 론 …………………………………………………………………………………………………………. 3
1. 실험 제목 …………………………………………………………………………………………….. 3
2. 실험 목적 …………………………………………………………………………………………….. 3
II. 본 론 …………………………………………………………………………………………………………. 3
1. 이론 및 원리 ……………………………………………………………………………………….. 4
2. 장치 및 방법 ……………………………………………………………………………………….. 4 
3. 실험 결과 …………………………………………………………………………………………….. 6
III. 결론 및 토의 …………………………………………………………………………………………… 10
IV. 참 고 문 헌 …………………………………………………………………………………………….. 12
"""
print(ori_text)

In [918]:
ori_text = """1 Introduction
In this paper, we present Mixtral 8x7B, a sparse mixture of experts model (SMoE) with open weights,
licensed under Apache 2.0. Mixtral outperforms Llama 2 70B and GPT-3.5 on most benchmarks. As
it only uses a subset of its parameters for every token, Mixtral allows faster inference speed at low
batch-sizes, and higher throughput at large batch-sizes.
Mixtral is a sparse mixture-of-experts network. It is a decoder-only model where the feedforward
block picks from a set of 8 distinct groups of parameters. At every layer, for every token, a router
network chooses two of these groups (the “experts”) to process the token and combine their output
additively. This technique increases the number of parameters of a model while controlling cost and
latency, as the model only uses a fraction of the total set of parameters per token.
Mixtral is pretrained with multilingual data using a context size of 32k tokens. It either matches
or exceeds the performance of Llama 2 70B and GPT-3.5, over several benchmarks. In particular,
arXiv:2401.04088v1 [cs.LG] 8 Jan 2024
Figure 1: Mixture of Experts Layer. Each input vector is assigned to 2 of the 8 experts by a router. The
layer’s output is the weighted sum of the outputs of the two selected experts. In Mixtral, an expert is a standard
feedforward block as in a vanilla transformer architecture.
Mixtral demonstrates superior capabilities in mathematics, code generation, and tasks that require
multilingual understanding, significantly outperforming Llama 2 70B in these domains. Experiments
show that Mixtral is able to successfully retrieve information from its context window of 32k tokens,
regardless of the sequence length and the location of the information in the sequence.
We also present Mixtral 8x7B – Instruct, a chat model fine-tuned to follow instructions using
supervised fine-tuning and Direct Preference Optimization [25]. Its performance notably surpasses
that of GPT-3.5 Turbo, Claude-2.1, Gemini Pro, and Llama 2 70B – chat model on human evaluation
benchmarks. Mixtral – Instruct also demonstrates reduced biases, and a more balanced sentiment
profile in benchmarks such as BBQ, and BOLD.
We release both Mixtral 8x7B and Mixtral 8x7B – Instruct under the Apache 2.0 license1
, free for
academic and commercial usage, ensuring broad accessibility and potential for diverse applications.
To enable the community to run Mixtral with a fully open-source stack, we submitted changes to
the vLLM project, which integrates Megablocks CUDA kernels for efficient inference. Skypilot also
allows the deployment of vLLM endpoints on any instance in the cloud.
2 Architectural details
Parameter Value
dim 4096
n_layers 32
head_dim 128
hidden_dim 14336
n_heads 32
n_kv_heads 8
context_len 32768
vocab_size 32000
num_experts 8
top_k_experts 2
Table 1: Model architecture.
Mixtral is based on a transformer architecture [31] and uses the same
modifications as described in [18], with the notable exceptions that Mixtral supports a fully dense context length of 32k tokens, and the feedforward blocks are replaced by Mixture-of-Expert layers (Section 2.1).
The model architecture parameters are summarized in Table 1.
2.1 Sparse Mixture of Experts
We present a brief overview of the Mixture of Experts layer (Figure 1).
For a more in-depth overview, see [12]. The output of the MoE module
for a given input x is determined by the weighted sum of the outputs
of the expert networks, where the weights are given by the gating
network’s output. i.e. given n expert networks {E0, Ei
, ..., En−1}, the
output of the expert layer is given by:
nX−1
i=0
G(x)i
· Ei(x).
Here, G(x)i denotes the n-dimensional output of the gating network for the i-th expert, and Ei(x)
is the output of the i-th expert network. If the gating vector is sparse, we can avoid computing
the outputs of experts whose gates are zero. There are multiple alternative ways of implementing
G(x) [6, 15, 35], but a simple and performant one is implemented by taking the softmax over the
Top-K logits of a linear layer [28]. We use
G(x) := Softmax(TopK(x · Wg)),
where (TopK(ℓ))i
:= ℓi
if ℓi
is among the top-K coordinates of logits ℓ ∈ R
n and (TopK(ℓ))i
:= −∞
otherwise. The value of K – the number of experts used per token – is a hyper-parameter that modulates the amount of compute used to process each token. If one increases n while keeping K fixed, one
1
https://mistral.ai/news/mixtral-of-experts/
2
can increase the model’s parameter count while keeping its computational cost effectively constant.
This motivates a distinction between the model’s total parameter count (commonly referenced as the
sparse parameter count), which grows with n, and the number of parameters used for processing an
individual token (called the active parameter count), which grows with K up to n.
MoE layers can be run efficiently on single GPUs with high performance specialized kernels. For
example, Megablocks [13] casts the feed-forward network (FFN) operations of the MoE layer as large
sparse matrix multiplications, significantly enhancing the execution speed and naturally handling
cases where different experts get a variable number of tokens assigned to them. Moreover, the
MoE layer can be distributed to multiple GPUs through standard Model Parallelism techniques, and
through a particular kind of partitioning strategy called Expert Parallelism (EP) [28]. During the MoE
layer’s execution, tokens meant to be processed by a specific expert are routed to the corresponding
GPU for processing, and the expert’s output is returned to the original token location. Note that EP
introduces challenges in load balancing, as it is essential to distribute the workload evenly across the
GPUs to prevent overloading individual GPUs or hitting computational bottlenecks.
In a Transformer model, the MoE layer is applied independently per token and replaces the
feed-forward (FFN) sub-block of the transformer block. For Mixtral we use the same SwiGLU
architecture as the expert function Ei(x) and set K = 2. This means each token is routed to two
SwiGLU sub-blocks with different sets of weights. Taking this all together, the output y for an input
token x is computed as:
y =
nX−1
i=0
Softmax(Top2(x · Wg))i
· SwiGLUi(x).
This formulation is similar to the GShard architecture [21], with the exceptions that we replace all
FFN sub-blocks by MoE layers while GShard replaces every other block, and that GShard uses a
more elaborate gating strategy for the second expert assigned to each token.
3 Results
We compare Mixtral to Llama, and re-run all benchmarks with our own evaluation pipeline for fair
comparison. We measure performance on a wide variety of tasks categorized as follow:
• Commonsense Reasoning (0-shot): Hellaswag [32], Winogrande [26], PIQA [3], SIQA [27],
OpenbookQA [22], ARC-Easy, ARC-Challenge [8], CommonsenseQA [30]
• World Knowledge (5-shot): NaturalQuestions [20], TriviaQA [19]
• Reading Comprehension (0-shot): BoolQ [7], QuAC [5]
• Math: GSM8K [9] (8-shot) with maj@8 and MATH [17] (4-shot) with maj@4
• Code: Humaneval [4] (0-shot) and MBPP [1] (3-shot)
• Popular aggregated results: MMLU [16] (5-shot), BBH [29] (3-shot), and AGI Eval [34]
(3-5-shot, English multiple-choice questions only)
Figure 2: Performance of Mixtral and different Llama models on a wide range of benchmarks. All models
were re-evaluated on all metrics with our evaluation pipeline for accurate comparison. Mixtral outperforms or
matches Llama 2 70B on all benchmarks. In particular, it is vastly superior in mathematics and code generation.
3
Model Active
Params MMLU HellaS WinoG PIQA Arc-e Arc-c NQ TriQA HumanE MBPP Math GSM8K
LLaMA 2 7B 7B 44.4% 77.1% 69.5% 77.9% 68.7% 43.2% 17.5% 56.6% 11.6% 26.1% 3.9% 16.0%
LLaMA 2 13B 13B 55.6% 80.7% 72.9% 80.8% 75.2% 48.8% 16.7% 64.0% 18.9% 35.4% 6.0% 34.3%
LLaMA 1 33B 33B 56.8% 83.7% 76.2% 82.2% 79.6% 54.4% 24.1% 68.5% 25.0% 40.9% 8.4% 44.1%
LLaMA 2 70B 70B 69.9% 85.4% 80.4% 82.6% 79.9% 56.5% 25.4% 73.0% 29.3% 49.8% 13.8% 69.6%
Mistral 7B 7B 62.5% 81.0% 74.2% 82.2% 80.5% 54.9% 23.2% 62.5% 26.2% 50.2% 12.7% 50.0%
Mixtral 8x7B 13B 70.6% 84.4% 77.2% 83.6% 83.1% 59.7% 30.6% 71.5% 40.2% 60.7% 28.4% 74.4%
Table 2: Comparison of Mixtral with Llama. Mixtral outperforms or matches Llama 2 70B performance on
almost all popular benchmarks while using 5x fewer active parameters during inference.
Figure 3: Results on MMLU, commonsense reasoning, world knowledge and reading comprehension,
math and code for Mistral (7B/8x7B) vs Llama 2 (7B/13B/70B). Mixtral largely outperforms Llama 2 70B
on all benchmarks, except on reading comprehension benchmarks while using 5x lower active parameters. It
is also vastly superior to Llama 2 70B on code and math.
Detailed results for Mixtral, Mistral 7B and Llama 2 7B/13B/70B and Llama 1 34B2
are reported
in Table 2. Figure 2 compares the performance of Mixtral with the Llama models in different
categories. Mixtral surpasses Llama 2 70B across most metrics. In particular, Mixtral displays a
superior performance in code and mathematics benchmarks.
Size and Efficiency. We compare our performance to the Llama 2 family, aiming to understand
Mixtral models’ efficiency in the cost-performance spectrum (see Figure 3). As a sparse Mixtureof-Experts model, Mixtral only uses 13B active parameters for each token. With 5x lower active
parameters, Mixtral is able to outperform Llama 2 70B across most categories.
Note that this analysis focuses on the active parameter count (see Section 2.1), which is directly
proportional to the inference compute cost, but does not consider the memory costs and hardware
utilization. The memory costs for serving Mixtral are proportional to its sparse parameter count,
47B, which is still smaller than Llama 2 70B. As for device utilization, we note that the SMoEs layer
introduces additional overhead due to the routing mechanism and due to the increased memory loads
when running more than one expert per device. They are more suitable for batched workloads where
one can reach a good degree of arithmetic intensity.
Comparison with Llama 2 70B and GPT-3.5. In Table 3, we report the performance of Mixtral 8x7B
compared to Llama 2 70B and GPT-3.5. We observe that Mixtral performs similarly or above the
two other models. On MMLU, Mixtral obtains a better performance, despite its significantly smaller
capacity (47B tokens compared to 70B). For MT Bench, we report the performance of the latest
GPT-3.5-Turbo model available, gpt-3.5-turbo-1106.
2
Since Llama 2 34B was not open-sourced, we report results for Llama 1 34B.
4
LLaMA 2 70B GPT-3.5 Mixtral 8x7B
MMLU
(MCQ in 57 subjects) 69.9% 70.0% 70.6%
HellaSwag
(10-shot) 87.1% 85.5% 86.7%
ARC Challenge
(25-shot) 85.1% 85.2% 85.8%
WinoGrande
(5-shot) 83.2% 81.6% 81.2%
MBPP
(pass@1) 49.8% 52.2% 60.7%
GSM-8K
(5-shot) 53.6% 57.1% 58.4%
MT Bench
(for Instruct Models) 6.86 8.32 8.30
Table 3: Comparison of Mixtral with Llama 2 70B and GPT-3.5. Mixtral outperforms or matches Llama 2
70B and GPT-3.5 performance on most metrics.
Evaluation Differences. On some benchmarks, there are some differences between our evaluation
protocol and the one reported in the Llama 2 paper: 1) on MBPP, we use the hand-verified subset 2)
on TriviaQA, we do not provide Wikipedia contexts.
3.1 Multilingual benchmarks
Compared to Mistral 7B, we significantly upsample the proportion of multilingual data during
pretraining. The extra capacity allows Mixtral to perform well on multilingual benchmarks while
maintaining a high accuracy in English. In particular, Mixtral significantly outperforms Llama 2 70B
in French, German, Spanish, and Italian, as shown in Table 4.
Active
Params
French German Spanish Italian
Model Arc-c HellaS MMLU Arc-c HellaS MMLU Arc-c HellaS MMLU Arc-c HellaS MMLU
LLaMA 1 33B 33B 39.3% 68.1% 49.9% 41.1% 63.3% 48.7% 45.7% 69.8% 52.3% 42.9% 65.4% 49.0%
LLaMA 2 70B 70B 49.9% 72.5% 64.3% 47.3% 68.7% 64.2% 50.5% 74.5% 66.0% 49.4% 70.9% 65.1%
Mixtral 8x7B 13B 58.2% 77.4% 70.9% 54.3% 73.0% 71.5% 55.4% 77.6% 72.5% 52.8% 75.1% 70.9%
Table 4: Comparison of Mixtral with Llama on Multilingual Benchmarks. On ARC Challenge, Hellaswag,
and MMLU, Mixtral outperforms Llama 2 70B on 4 languages: French, German, Spanish, and Italian.
3.2 Long range performance
To assess the capabilities of Mixtral to tackle long context, we evaluate it on the passkey retrieval
task introduced in [23], a synthetic task designed to measure the ability of the model to retrieve a
passkey inserted randomly in a long prompt. Results in Figure 4 (Left) show that Mixtral achieves a
100% retrieval accuracy regardless of the context length or the position of passkey in the sequence.
Figure 4 (Right) shows that the perplexity of Mixtral on a subset of the proof-pile dataset [2] decreases
monotonically as the size of the context increases.
Figure 4: Long range performance of Mixtral. (Left) Mixtral has 100% retrieval accuracy of the Passkey task
regardless of the location of the passkey and length of the input sequence. (Right) The perplexity of Mixtral on
the proof-pile dataset decreases monotonically as the context length increases.
5
3.3 Bias Benchmarks
Llama 2 70B Mixtral 8x7B
BBQ accuracy 51.5% 56.0%
BOLD sentiment score (avg ± std)
gender 0.293 ± 0.073 0.323 ±0.045
profession 0.218 ± 0.073 0.243 ± 0.087
religious_ideology 0.188 ± 0.133 0.144 ± 0.089
political_ideology 0.149 ± 0.140 0.186 ± 0.146
race 0.232 ± 0.049 0.232 ± 0.052
Figure 5: Bias Benchmarks. Compared Llama 2 70B,
Mixtral presents less bias (higher accuracy on BBQ, lower
std on BOLD) and displays more positive sentiment (higher
avg on BOLD).
To identify possible flaws to be corrected
by fine-tuning / preference modeling, we
measure the base model performance on
Bias Benchmark for QA (BBQ) [24] and
Bias in Open-Ended Language Generation
Dataset (BOLD) [10]. BBQ is a dataset
of hand-written question sets that target
attested social biases against nine different socially-relevant categories: age, disability status, gender identity, nationality,
physical appearance, race/ethnicity, religion,
socio-economic status, sexual orientation.
BOLD is a large-scale dataset that consists
of 23,679 English text generation prompts
for bias benchmarking across five domains.
We benchmark Llama 2 and Mixtral on BBQ and BOLD with our evaluation framework and report
the results in Table 5. Compared to Llama 2, Mixtral presents less bias on the BBQ benchmark
(56.0% vs 51.5%). For each group in BOLD, a higher average sentiment score means more positive
sentiments and a lower standard deviation indicates less bias within the group. Overall, Mixtral
displays more positive sentiments than Llama 2, with similar variances within each group.
4 Instruction Fine-tuning
We train Mixtral – Instruct using supervised fine-tuning (SFT) on an instruction dataset followed by
Direct Preference Optimization (DPO) [25] on a paired feedback dataset. Mixtral – Instruct reaches a
score of 8.30 on MT-Bench [33] (see Table 2), making it the best open-weights model as of December
2023. Independent human evaluation conducted by LMSys is reported in Figure 6
3
and shows that
Mixtral – Instruct outperforms GPT-3.5-Turbo, Gemini Pro, Claude-2.1, and Llama 2 70B chat.
Figure 6: LMSys Leaderboard. (Screenshot from Dec 22, 2023) Mixtral 8x7B Instruct v0.1 achieves an Arena
Elo rating of 1121 outperforming Claude-2.1 (1117), all versions of GPT-3.5-Turbo (1117 best), Gemini Pro
(1111), and Llama-2-70b-chat (1077). Mixtral is currently the best open-weights model by a large margin.
3
https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard
6
5 Routing analysis
In this section, we perform a small analysis on the expert selection by the router. In particular,
we are interested to see if during training some experts specialized to some specific domains (e.g.
mathematics, biology, philosophy, etc.).
To investigate this, we measure the distribution of selected experts on different subsets of The Pile
validation dataset [14]. Results are presented in Figure 7, for layers 0, 15, and 31 (layers 0 and 31
respectively being the first and the last layers of the model). Surprisingly, we do not observe obvious
patterns in the assignment of experts based on the topic. For instance, at all layers, the distribution of
expert assignment is very similar for ArXiv papers (written in Latex), for biology (PubMed Abstracts),
and for Philosophy (PhilPapers) documents.
Only for DM Mathematics we note a marginally different distribution of experts. This divergence is
likely a consequence of the dataset’s synthetic nature and its limited coverage of the natural language
spectrum, and is particularly noticeable at the first and last layers, where the hidden states are very
correlated to the input and output embeddings respectively.
This suggests that the router does exhibit some structured syntactic behavior. Figure 8 shows
examples of text from different domains (Python code, mathematics, and English), where each token
is highlighted with a background color corresponding to its selected expert. The figure shows that
words such as ‘self’ in Python and ‘Question’ in English often get routed through the same expert
even though they involve multiple tokens. Similarly, in code, the indentation tokens are always
assigned to the same experts, particularly at the first and last layers where the hidden states are more
correlated to the input and output of the model.
We also note from Figure 8 that consecutive tokens are often assigned the same experts. In fact, we
observe some degree of positional locality in The Pile datasets. Table 5 shows the proportion of consecutive tokens that get the same expert assignments per domain and layer. The proportion of repeated
0
0.05
0.10
0.15
0.20
layer: 0
0
0.05
0.10
0.15
0.20
layer: 15
0 1 2 3 4 5 6 7
0
0.05
0.10
0.15
0.20
layer: 31
Expert ID
Selection proportion
ArXiv
DM Mathematics
Github
Gutenberg
PhilPapers
PubMed Abstracts
StackExchange
Wikipedia (en)
Figure 7: Proportion of tokens assigned to each expert on different domains from The Pile dataset for
layers 0, 15, and 31. The gray dashed vertical line marks 1/8, i.e. the proportion expected with uniform
sampling. Here, we consider experts that are either selected as a first or second choice by the router. A
breakdown of the proportion of assignments done in each case cane be seen in Figure 9 in the Appendix.
7
First choice First or second choice
Layer 0 Layer 15 Layer 31 Layer 0 Layer 15 Layer 31
ArXiv 14.0% 27.9% 22.7% 46.5% 62.3% 52.9%
DM Mathematics 14.1% 28.4% 19.7% 44.9% 67.0% 44.5%
Github 14.9% 28.1% 19.7% 49.9% 66.9% 49.2%
Gutenberg 13.9% 26.1% 26.3% 49.5% 63.1% 52.2%
PhilPapers 13.6% 25.3% 22.1% 46.9% 61.9% 51.3%
PubMed Abstracts 14.2% 24.6% 22.0% 48.6% 61.6% 51.8%
StackExchange 13.6% 27.2% 23.6% 48.2% 64.6% 53.6%
Wikipedia (en) 14.4% 23.6% 25.3% 49.8% 62.1% 51.8%
Table 5: Percentage of expert assignment repetitions. We evaluate the proportion of times the same expert is
assigned to a token i and its following token i+1. We report whether the first chosen expert is the same, or whether
the same expert is observed as first or second choice in consecutive tokens. For reference, the expected proportion
of repetitions in the case of random assignments is 1
8 = 12.5% for “First choice” and 1 −
6
8
5
7 ≈ 46% for “First
and second choice”. Repetitions at the first layer are close to random, but are significantly higher at layers 15
and 31. The high number of repetitions shows that expert choice exhibits high temporal locality at these layers.
consecutive assignments is significantly higher than random for higher layers. This has implications
in how one might optimize the model for fast training and inference. For example, cases with high
locality are more likely to cause over-subscription of certain experts when doing Expert Parallelism.
Conversely, this locality can be leveraged for caching, as is done in [11]. A more complete view of
these same expert frequency is provided for all layers and across datasets in Figure 10 in the Appendix.
6 Conclusion
In this paper, we introduced Mixtral 8x7B, the first mixture-of-experts network to reach a state-of-theart performance among open-source models. Mixtral 8x7B Instruct outperforms Claude-2.1, Gemini Pro, and GPT-3.5 Turbo on human evaluation benchmarks. Because it only uses two experts at each
time step, Mixtral only uses 13B active parameters per token while outperforming the previous best
model using 70B parameters per token (Llama 2 70B). We are making our trained and fine-tuned models publicly available under the Apache 2.0 license. By sharing our models, we aim to facilitate the development of new techniques and applications that can benefit a wide range of industries and domains.
Figure 8: Text samples where each token is colored with the first expert choice. The selection of experts
appears to be more aligned with the syntax rather than the domain, especially at the initial and final layers.
8
Acknowledgements
We thank the CoreWeave and Scaleway teams for technical support as we trained our models. We
are grateful to NVIDIA for supporting us in integrating TensorRT-LLM and Triton and working
alongside us to make a sparse mixture of experts compatible with TensorRT-LLM.
"""
# print(ori_text)

In [895]:
ori_text = """1. 실험제목
등가속도운동 가속도 측정 실험

 

2. 실험목적
영상분석시스템을 이용하여 마찰이 없는 에어트랙에서 추에 매달린 물체의 운동을 통해 뉴턴의 운동 2법칙을 확인한다


3. 실험결과

 	1회	2회	3회
추(추걸이 포함)
질량(m₁) (g)	14.81	14.81	25.29
글라이더(연결자 포함)
질량(m₂) (g)	192.17	212.15	212.15
T-X그래프의 추세식(R²값 포함)	x = -32.365(t²) - 1.6224t + 122.68
R² = 1	x = -29.196(t²) - 1.1762t + 121.74
R² = 1	x = -48.747(t²) - 3.0515t + 120.9
R² = 1
가속도-측정치(a’) (cm/s²)	64.73	58.392	97.494
가속도-계산치(a) (cm/s²)	70.12	63.94	104.38
상대오차 |1-a’/a| * 100	7.686	8.676	6.597
 

4. 실험결과의 분석

일정한 가속도로 직선 운동하는 물체의 운동을 등가속도 운동이라 하며 뉴턴의 운동 2법칙을 이용하면 일정한 힘 ( 을 받는 질량 ( 인 물체의 가속도를 구할 수 있다.

 

일정 힘이 작용하는 경우 속도를 시간의 함수로 나타내면 v(t) = v0 + at 가 나오고 이 경우 질점의 위치 x 는 x(t) = x0 + v0t + (1/2)at² 이며 실험에서 얻은 질점의 추세식이 x(t) =C + Bt + At² 일 때 , 실험에 의한 질점의 가속도 a’ = 2A 를 구할 수 있었다 .


추와 추걸이의 무게가 m₁, 글라이더의 총 질량이 m₂ 일 때 (m₁ + m₂)a = m₁g 계산을 통해 가속도 a = m₁g / m₁ + m₂ 를 구할 수 있었다 .

 

실험결과에서 가속도 측정치는 가속도 계산치에 비해 더 적은 값을 나타냈는데 이는 글라이더의 마찰이나 공기저항 등으로 인한 결과로 보인다 . 또한 세 실험의 상대오차가 10미만으로 적은 값으로 나와 전체적으로 뉴턴의 제 2법칙이 잘 적용된 것으로 보인다.


실험2 에서는 실험 1 에서의 글라이더의 무게에 10g 추가하여 실험을 진행했다 . 글라이더의 질량이 증가하였으므로 마찰력이 증가하여 가속도가 실험 1 에 비해 줄어들었고 상대오차가 소폭 커진 것을 볼 수 있다.

 

실험3 에서는 추의 무게를 10g 추가하여 가속도를 증가시켰다 . 가속도 측정치와 계산치가 크게 증가하였고 상대오차는 실험 1, 2 와 비교하여 줄어든 것으로 보아 외부요인의 방해 없이 실험이 잘 진행되었음을 알 수 있다
"""

In [947]:
ori_text = """1 INTRODUCTION
“The limits of my language mean the limits of my world.”
—Ludwig Wittgenstein
LANGUAGE is a prominent ability in human beings to
express and communicate, which develops in early
childhood and evolves over a lifetime [3, 4]. Machines,
however, cannot naturally grasp the abilities of understanding and communicating in the form of human language,
unless equipped with powerful artificial intelligence (AI)
algorithms. It has been a longstanding research challenge
to achieve this goal, to enable machines to read, write, and
communicate like humans [5].
Technically, language modeling (LM) is one of the major
approaches to advancing language intelligence of machines.
In general, LM aims to model the generative likelihood
of word sequences, so as to predict the probabilities of
future (or missing) tokens. The research of LM has received
extensive attention in the literature, which can be divided
into four major development stages:
• Statistical language models (SLM). SLMs [6–9] are developed based on statistical learning methods that rose in
the 1990s. The basic idea is to build the word prediction
model based on the Markov assumption, e.g., predicting the
next word based on the most recent context. The SLMs with
a fixed context length n are also called n-gram language
models, e.g., bigram and trigram language models. SLMs
have been widely applied to enhance task performance
in information retrieval (IR) [10, 11] and natural language
processing (NLP) [12–14]. However, they often suffer from
the curse of dimensionality: it is difficult to accurately
estimate high-order language models since an exponential
number of transition probabilities need to be estimated.
Thus, specially designed smoothing strategies such as backoff estimation [15] and Good–Turing estimation [16] have
been introduced to alleviate the data sparsity problem.
• Neural language models (NLM). NLMs [1, 17, 18] characterize the probability of word sequences by neural networks,
e.g., multi-layer perceptron (MLP) and recurrent neural networks (RNNs). As a remarkable contribution, the work in
[1] introduced the concept of distributed representation of
words and built the word prediction function conditioned
on the aggregated context features (i.e., the distributed
word vectors). By extending the idea of learning effective
features for text data, a general neural network approach
Fig. 1: The trends of the cumulative numbers of arXiv papers that contain the keyphrases “language model” (since June 2018)
and “large language model” (since October 2019), respectively. The statistics are calculated using exact match by querying
the keyphrases in title or abstract by months. We set different x-axis ranges for the two keyphrases, because “language
models” have been explored at an earlier time. We label the points corresponding to important landmarks in the research
progress of LLMs. A sharp increase occurs after the release of ChatGPT: the average number of published arXiv papers
that contain “large language model” in title or abstract goes from 0.40 per day to 8.58 per day (Figure 1(b)).
Fig. 2: An evolution process of the four generations of language models (LM) from the perspective of task solving capacity.
Note that the time period for each stage may not be very accurate, and we set the time mainly according to the publish
date of the most representative studies at each stage. For neural language models, we abbreviate the paper titles of
two representative studies to name the two approaches: NPLM [1] (“A neural probabilistic language model”) and NLPS [2]
(“Natural language processing (almost) from scratch”). Due to the space limitation, we don’t list all representative studies in
this figure.
was developed to build a unified, end-to-end solution for
various NLP tasks [2]. Furthermore, word2vec [19, 20] was
proposed to build a simplified shallow neural network
for learning distributed word representations, which were
demonstrated to be very effective across a variety of NLP
tasks. These studies have initiated the use of language
models for representation learning (beyond word sequence
modeling), having an important impact on the field of NLP.
• Pre-trained language models (PLM). As an early attempt, ELMo [21] was proposed to capture context-aware
word representations by first pre-training a bidirectional
LSTM (biLSTM) network (instead of learning fixed word
representations) and then fine-tuning the biLSTM network
according to specific downstream tasks. Furthermore, based
on the highly parallelizable Transformer architecture [22]
with self-attention mechanisms, BERT [23] was proposed by
pre-training bidirectional language models with specially
designed pre-training tasks on large-scale unlabeled corpora. These pre-trained context-aware word representations
are very effective as general-purpose semantic features,
which have largely raised the performance bar of NLP
tasks. This study has inspired a large number of follow-up
work, which sets the “pre-training and fine-tuning” learning
paradigm. Following this paradigm, a great number of studies on PLMs have been developed, introducing either different architectures [24, 25] (e.g., GPT-2 [26] and BART [24]) or
improved pre-training strategies [27–29]. In this paradigm, it
often requires fine-tuning the PLM for adapting to different
downstream tasks.
• Large language models (LLM). Researchers find that
scaling PLM (e.g., scaling model size or data size) often
leads to an improved model capacity on downstream tasks
(i.e., following the scaling law [30]). A number of studies
have explored the performance limit by training an ever
larger PLM (e.g., the 175B-parameter GPT-3 and the 540Bparameter PaLM). Although scaling is mainly conducted
in model size (with similar architectures and pre-training
tasks), these large-sized PLMs display different behaviors
from smaller PLMs (e.g., 330M-parameter BERT and 1.5Bparameter GPT-2) and show surprising abilities (called emergent abilities [31]) in solving a series of complex tasks. For
example, GPT-3 can solve few-shot tasks through in-context
learning, whereas GPT-2 cannot do well. Thus, the research
community coins the term “large language models (LLM)”
1
for these large-sized PLMs [32–35], which attract increasing
research attention (See Figure 1). A remarkable application
of LLMs is ChatGPT2
that adapts the LLMs from the GPT
series for dialogue, which presents an amazing conversation
ability with humans. We can observe a sharp increase of the
arXiv papers that are related to LLMs after the release of
ChatGPT in Figure 1.
As discussed before, language model is not a new technical concept specially for LLMs, but has evolved with the
advance of artificial intelligence over the decades. Early language models mainly aim to model and generate text data,
while latest language models (e.g., GPT-4) focus on complex
task solving. From language modeling to task solving, it is an
important leap in scientific thinking, which is the key to
understand the development of language models in the research history. From the perspective of task solving, the four
generations of language models have exhibited different levels of model capacities. In Figure 2, we describe the evolution process of language models in terms of the task solving
capacity. At first, statistical language models mainly assisted
in some specific tasks (e.g., retrieval or speech tasks), in
which the predicted or estimated probabilities can enhance
the performance of task-specific approaches. Subsequently,
neural language models focused on learning task-agnostic
representations (e.g., features), aiming to reduce the efforts
for human feature engineering. Furthermore, pre-trained
language models learned context-aware representations that
can be optimized according to downstream tasks. For the
latest generation of language model, LLMs are enhanced by
exploring the scaling effect on model capacity, which can be
considered as general-purpose task solvers. To summarize,
in the evolution process, the task scope that can be solved
by language models have been greatly extended, and the
task performance attained by language models have been
significantly enhanced.
In the existing literature, PLMs have been widely discussed and surveyed [36–39], while LLMs are seldom reviewed in a systematic way. To motivate our survey, we first
highlight three major differences between LLMs and PLMs.
First, LLMs display some surprising emergent abilities that
may not be observed in previous smaller PLMs. These abilities are key to the performance of language models on complex tasks, making AI algorithms unprecedently powerful
and effective. Second, LLMs would revolutionize the way
that humans develop and use AI algorithms. Unlike small
PLMs, the major approach to accessing LLMs is through
the prompting interface (e.g., GPT-4 API). Humans have to
understand how LLMs work and format their tasks in a way
that LLMs can follow. Third, the development of LLMs no
longer draws a clear distinction between research and engineering. The training of LLMs requires extensive practical
experiences in large-scale data processing and distributed
parallel training. To develop capable LLMs, researchers
have to solve complicated engineering issues, working with
engineers or being engineers.
Nowadays, LLMs are posing a significant impact on
the AI community, and the advent of ChatGPT and GPT-4
leads to the rethinking of the possibilities of artificial general
intelligence (AGI). OpenAI has published a technical article
entitled “Planning for AGI and beyond”, which discusses
the short-term and long-term plans to approach AGI [40],
and a more recent paper has argued that GPT-4 might be
considered as an early version of an AGI system [41]. The
research areas of AI are being revolutionized by the rapid
progress of LLMs. In the field of NLP, LLMs can serve as a
general-purpose language task solver (to some extent), and
the research paradigm has been shifting towards the use
of LLMs. In the field of IR, traditional search engines are
challenged by the new information seeking way through AI
chatbots (i.e., ChatGPT), and New Bing3 presents an initial
attempt that enhances the search results based on LLMs. In
the field of CV, the researchers try to develop ChatGPT-like
vision-language models that can better serve multimodal
dialogues [42–45], and GPT-4 [46] has supported multimodal input by integrating the visual information. This new
wave of technology would potentially lead to a prosperous
ecosystem of real-world applications based on LLMs. For
instance, Microsoft 365 is being empowered by LLMs (i.e.,
Copilot) to automate the office work, and OpenAI supports
the use of plugins in ChatGPT for implementing special
functions.
Despite the progress and impact, the underlying principles of LLMs are still not well explored. Firstly, it is
mysterious why emergent abilities occur in LLMs, instead of
smaller PLMs. As a more general issue, there lacks a deep,
detailed investigation of the key factors that contribute to
the superior abilities of LLMs. It is important to study when
and how LLMs obtain such abilities [47]. Although there are
some meaningful discussions about this problem [31, 47],
more principled investigations are needed to uncover the
“secrets“ of LLMs. Secondly, it is difficult for the research
community to train capable LLMs. Due to the huge demand of computation resources, it is very costly to carry
out repetitive, ablating studies for investigating the effect
of various strategies for training LLMs. Indeed, LLMs are
mainly trained by industry, where many important training
details (e.g., data collection and cleaning) are not revealed
to the public. Thirdly, it is challenging to align LLMs with
human values or preferences. Despite the capacities, LLMs
are also likely to produce toxic, fictitious, or harmful contents. It requires effective and efficient control approaches
to eliminating the potential risk of the use of LLMs [46].
Faced with both opportunities and challenges, it needs
more attention on the research and development of LLMs. In
order to provide a basic understanding of LLMs, this survey
conducts a literature review of the recent advances in LLMs
from four major aspects, including pre-training (how to pretrain a capable LLM), adaptation (how to effectively adapt
pre-trained LLMs for better use), utilization (how to use
LLMs for solving various downstream tasks) and capability
evaluation (how to evaluate the abilities of LLMs and existing
empirical findings). We thoroughly comb the literature and
summarize the key findings, techniques, and methods of
LLMs. For this survey, we also create a GitHub project
website by collecting the supporting resources for LLMs, at
the link https://github.com/RUCAIBox/LLMSurvey. We
are also aware of several related review articles on PLMs
or LLMs [32, 36, 38, 39, 43, 48–54]. These papers either
discuss PLMs or some specific (or general) aspects of LLMs.
Compared with them, we focus on the techniques and
methods to develop and use LLMs and provide a relatively
comprehensive reference to important aspects of LLMs.
The remainder of this survey is organized as follows:
Section 2 introduces the background for LLMs and the evolution of GPT-series models, followed by the summarization
of available resources for developing LLMs in Section 3.
Sections 4, 5, 6, and 7 review and summarize the recent
progress from the four aspects of pre-training, adaptation,
utilization, and capacity evaluation, respectively. Then, Section 8 discusses the practical guide for prompt design,
and Section 9 reviews the applications of LLMs in several
representative domains. Finally, we conclude the survey in
Section 10 by summarizing the major findings and discuss
the remaining issues for future work.
2 OVERVIEW
In this section, we present an overview about the background of LLMs and then summarize the technical evolution of the GPT-series models.
2.1 Background for LLMs
Typically, large language models (LLMs) refer to Transformer
language models that contain hundreds of billions (or
more) of parameters4
, which are trained on massive text
data [32], such as GPT-3 [55], PaLM [56], Galactica [35],
and LLaMA [57]. LLMs exhibit strong capacities to understand natural language and solve complex tasks (via
text generation). To have a quick understanding of how
LLMs work, this part introduces the basic background for
LLMs, including scaling laws, emergent abilities and key
techniques.
Formulation of Scaling Laws for LLMs. Currently, LLMs
are mainly built upon the Transformer architecture [22],
where multi-head attention layers are stacked in a very
deep neural network. Existing LLMs adopt similar Transformer architectures and pre-training objectives (e.g., language modeling) as small language models. However, LLMs
significantly extend the model size, data size, and total
compute (orders of magnification). Extensive research has
shown that scaling can largely improve the model capacity
of LLMs [26, 55, 56]. Thus, it is useful to establish a quantitative approach to characterizing the scaling effect. Next, we
introduce two representative scaling laws for Transformer
language models [30, 34].
• KM scaling law5
. In 2020, Kaplan et al. [30] (the OpenAI
team) firstly proposed to model the power-law relationship
of model performance with respective to three major factors,
namely model size (N), dataset size (D), and the amount of
training compute (C), for neural language models. Given
a compute budget c, they empirically presented three basic
formulas for the scaling law6
:
L(N) = 
Nc
N
αN
, αN ∼ 0.076, Nc ∼ 8.8 × 1013 (1)
L(D) = 
Dc
D
αD
, αD ∼ 0.095, Dc ∼ 5.4 × 1013
L(C) = 
Cc
C
αC
, αC ∼ 0.050, Cc ∼ 3.1 × 108
where L(·) denotes the cross entropy loss in nats, and
a follow-up study [58] from OpenAI has shown that the
language modeling loss can be decomposed into two parts,
namely irreducible loss (the entropy of the true data distribution) and reducible loss (an estimate of the KL divergence
between the true and model distributions). The three laws
were derived by fitting the model performance with varied
data sizes (22M to 23B tokens), model sizes (768M to 1.5B
non-embedding parameters) and training compute, under
some assumptions (e.g., the analysis of one factor should
be not bottlenecked by the other two factors). They showed
that the model performance has a strong dependence relation on the three factors.
• Chinchilla scaling law. As another representative study,
Hoffmann et al. [34] (the Google DeepMind team) proposed
an alternative form for scaling laws to instruct the computeoptimal training for LLMs. They conducted rigorous experiments by varying a larger range of model sizes (70M to
16B) and data sizes (5B to 500B tokens), and fitted a similar
scaling law yet with different coefficients as below [34]:
L(N, D) = E +
A
Nα
+
B
Dβ
, (2)
where E = 1.69, A = 406.4, B = 410.7, α = 0.34 and
β = 0.28. By optimizing the loss L(N, D) under the constraint C ≈ 6ND, they showed that the optimal allocation
of compute budget to model size and data size can be
derived as follows:
Nopt(C) = G

C
6
a
, Dopt(C) = G
−1

C
6
b
, (3)
where a =
α
α+β
, b =
β
α+β
and G is a scaling coefficient that
can be computed by A, B, α and β. As analyzed in [34],
given an increase in compute budget, the KM scaling law
favors a larger budget allocation in model size than the data
size, while the Chinchilla scaling law argues that the two
sizes should be increased in equal scales, i.e., having similar
values for a and b in Equation (3).
Discussion on Scaling Laws. After introducing the formulations, we continue to discuss scaling law in the following
two aspects, to enhance its understanding:
• Predictable scaling. In practice, scaling law can be used
to instruct the training of LLMs, and it has been proven
feasible to reliably estimate the performance of larger models based on that of smaller models, called predictable scaling [46]. The benefits of predictable scaling for training
LLMs are mainly twofold. Firstly, for large models, it is
infeasible to rigorously examine various training tricks or
variants, and it would be very helpful if experiences gained
from small models could also apply to large models. For
instance, small proxy models can be trained to find the
optimal schedule of the data mixture for large models [59].
Secondly, the training of large-scale models takes a long
time, often suffering from issues such as training loss spike,
and scaling law can be employed to monitor the training
status of LLMs, e.g., identifying abnormal performance at an
early time. Despite that scaling law characterizes a smooth
trend of performance increase (or loss decrease), it also
indicates that diminishing returns7 might occur as model
scaling. An empirical study [58] from the OpenAI team
has shown that representation quality or semantic content
can still effectively improve even if approaching the point
of diminishing returns (i.e., approaching the irreducible
loss) [58]. This finding suggests that training large models
are promising for improving the performance of downstream tasks. To further explore scaling effect, a potential
issue is that the amount of available data for training LLMs
is actually limited. With the ever-increasing model scale, the
public text data would be soon “exhausted” for LLMs [60].
Thus, it will be meaningful to study how scaling laws apply
to a data-constrained regime [61], where data repetition or
augmentation might be useful to alleviate data scarcity.
• Task-level predictability. Existing research of scaling laws
are mostly conducted in terms of language modeling loss
(e.g., per-token cross-entropy loss in nats [30]), while in
practice we are more concerned about the performance of
LLMs on actual tasks. Thus, a basic problem is that how
the decrease of language modeling loss translates into the
improvement of task performance [58]. Intuitively, a model
with a smaller language modeling loss tends to yield a
better performance on downstream tasks, since language
modeling loss can be considered as a general measure of
the overall model capacity. GPT-4 [46] has reported that
some capabilities (e.g., coding ability) can be accurately
predicted via scaling law. Despite that, readers should be
aware that a direct decrease in language modeling loss does
not always indicate an improvement of model performance
on downstream tasks. Specially, the phenomenon of inverse
scaling would occur for some tasks, where task performance
surprisingly becomes worse as the language modeling loss
decreases [62]. Overall, it is more difficult to explore and
characterize task-level scaling laws, since it might be also
dependent on task-related information (task metric, task
difficulty, etc.). Furthermore, some capacities (e.g., in-context
learning [55]) are unpredictable according to the scaling law,
which can be observed only when the model size exceeds a
certain level (as discussed below).
Emergent Abilities of LLMs. In the literature [31], emergent
abilities of LLMs are formally defined as “the abilities that
are not present in small models but arise in large models”,
which is one of the most prominent features that distinguish LLMs from previous PLMs. It further introduces a
notable characteristic when emergent abilities occur [31]:
performance rises significantly above random when the
scale reaches a certain level. By analogy, such an emergent
pattern has close connections with the phenomenon of phase
transition in physics [31, 63]. In principle, emergent abilities
can be defined in relation to some complex tasks [31, 64],
while we are more concerned with general abilities that
can be applied to solve a variety of tasks. Here, we briefly
introduce three typical emergent abilities for LLMs and
representative models that possess such an ability8
.
• In-context learning. The in-context learning (ICL) ability
is formally introduced by GPT-3 [55]: assuming that the
language model has been provided with a natural language
instruction and/or several task demonstrations, it can generate the expected output for the test instances by completing the word sequence of input text, without requiring
additional training or gradient update9
. Among the GPTseries models, the 175B GPT-3 model exhibited a strong ICL
ability in general, but not the GPT-1 and GPT-2 models. Such
an ability also depends on the specific downstream task. For
example, the ICL ability can emerge on the arithmetic tasks
(e.g., the 3-digit addition and subtraction) for the 13B GPT-3,
but 175B GPT-3 even cannot work well on the Persian QA
task [31].
• Instruction following. By fine-tuning with a mixture of
multi-task datasets formatted via natural language descriptions (called instruction tuning), LLMs are shown to perform
well on unseen tasks that are also described in the form
of instructions [28, 66, 67]. With instruction tuning, LLMs
are enabled to follow the task instructions for new tasks
without using explicit examples, thus having an improved
generalization ability. According to the experiments in [67],
instruction-tuned LaMDA-PT [68] started to significantly
outperform the untuned one on unseen tasks when the
model size reached 68B, but not for 8B or smaller model
sizes. A recent study [69] found that a model size of 62B is
at least required for PaLM to perform well on various tasks
in four evaluation benchmarks (i.e., MMLU, BBH, TyDiQA
and MGSM), though a much smaller size might suffice for
some specific tasks (e.g., MMLU).
• Step-by-step reasoning. For small language models, it
is usually difficult to solve complex tasks that involve
multiple reasoning steps, e.g., mathematical word problems.
In contrast, with the chain-of-thought (CoT) prompting
strategy [33], LLMs can solve such tasks by utilizing the
prompting mechanism that involves intermediate reasoning
steps for deriving the final answer. This ability is speculated
to be potentially obtained by training on code [33, 47]. An
empirical study [33] has shown that CoT prompting can
bring performance gains (on arithmetic reasoning benchmarks) when applied to PaLM and LaMDA variants with
a model size larger than 60B, while its advantage over
the standard prompting becomes more evident when the
model size exceeds 100B. Furthermore, the performance
improvement with CoT prompting seems to be also varied
for different tasks, e.g., GSM8K > MAWPS > SWAMP for
PaLM [33].
How Emergent Abilities Relate to Scaling Laws. In existing
literature [30, 31, 34], scaling laws and emergent abilities
provide two perspectives to understand the advantage of
large models over small models. In general, scaling law
(often measured by language modeling loss) describes predictable performance relation with the potential effect of
diminishing returns, while emergent abilities (often measured by task performance) are unpredictable but very profitable once such abilities actually emerge. Since the two
perspectives reflect different performance trends (continuous improvement v.s. sharp performance leap), they might
lead to misaligned findings or observations. There are also
extensive debates on the rationality of emergent abilities.
A popular speculation is that emergent abilities might be
partially attributed to the evaluation setting for special tasks
(e.g., the discontinuous evaluation metrics) [70, 71]: when
evaluation metrics are altered accordingly, the sharpness of
the emergent ability curve would disappear. However, the
performance of LLMs on most tasks are perceived by users
naturally in a discontinuous way. For instance, end users
prefer a reliable code generated by LLMs that can successfully pass the test case, but are less interested in selecting a
better code with fewer errors between two failed ones. More
recently, a study [72] proposes a new evaluation setting
that can enlarge the resolution of task metrics, making task
performance more predictable. Despite these efforts, more
fundamental research (e.g., grokking10) about the working
mechanism of LLMs is still in need to understand the emergence of certain abilities. The subtle relation between scaling
law and emergent abilities can be explained by analogy with
the ability acquisition of human11. Take the speaking ability
as an example. For children, language development (especially infants) can be also considered as a multi-level process
where “emergent abilities” occur. Specially, the language
ability would relatively stable within a time interval, but
qualitative change only occurs when evolving into another
ability level (e.g., from speaking simple words to speaking
simple sentences). Such a learning process is essentially not
smooth and stable (i.e., language ability does not develop at
a constant rate over time), though a child actually grows
every day. It is interesting that young parents would be often
surprised by unexpected progress of the speaking ability
exhibited by their babies.
Key Techniques for LLMs. It has been a long way that
LLMs evolve into the current state: general and capable
learners. In the development process, a number of important techniques are proposed, which largely improve the
capacity of LLMs. Here, we briefly list several important
techniques that (potentially) lead to the success of LLMs, as
follows.
• Scaling. As discussed in previous parts, there exists
an evident scaling effect in Transformer language models: larger model/data sizes and more training compute
typically lead to an improved model capacity [30, 34]. As
two representative models, GPT-3 and PaLM explored the
scaling limits by increasing the model size to 175B and
540B, respectively. Since compute budget is usually limited,
scaling laws can be further employed to conduct a more
compute-efficient allocation of the compute resources. For
example, Chinchilla (with more training tokens) outperforms its counterpart model Gopher (with a larger model
size) by increasing the data scale with the same compute
budget [34]. In addition, data scaling should be with careful
cleaning process, since the quality of pre-training data plays
a key role in the model capacity.
• Training. Due to the huge model size, it is very challenging to successfully train a capable LLM. Distributed
training algorithms are needed to learn the network parameters of LLMs, in which various parallel strategies are often jointly utilized. To support distributed training, several
optimization frameworks have been released to facilitate
the implementation and deployment of parallel algorithms,
such as DeepSpeed [74] and Megatron-LM [75–77]. Also, optimization tricks are also important for training stability and
model performance, e.g., restart to overcome training loss
spike [56] and mixed precision training [78]. More recently,
GPT-4 [46] proposes to develop special infrastructure and
optimization methods that reliably predict the performance
of large models with much smaller models.
• Ability eliciting. After being pre-trained on large-scale
corpora, LLMs are endowed with potential abilities as
general-purpose task solvers. These abilities might not be
explicitly exhibited when LLMs perform some specific tasks.
As the technical approach, it is useful to design suitable task
instructions or specific in-context learning strategies to elicit
such abilities. For instance, chain-of-thought prompting has
been shown to be useful to solve complex reasoning tasks
by including intermediate reasoning steps. Furthermore,
we can perform instruction tuning on LLMs with task
descriptions expressed in natural language, for improving
the generalizability of LLMs on unseen tasks. These eliciting
techniques mainly correspond to the emergent abilities of
LLMs, which may not show the same effect on small language models.
• Alignment tuning. Since LLMs are trained to capture
the data characteristics of pre-training corpora (including
both high-quality and low-quality data), they are likely to
generate toxic, biased, or even harmful content for humans.
It is necessary to align LLMs with human values, e.g., helpful,
honest, and harmless. For this purpose, InstructGPT [66]
7
designs an effective tuning approach that enables LLMs to
follow the expected instructions, which utilizes the technique of reinforcement learning with human feedback [66, 79].
It incorporates human in the training loop with elaborately
designed labeling strategies. ChatGPT is indeed developed
on a similar technique to InstructGPT, which shows a strong
alignment capacity in producing high-quality, harmless responses, e.g., rejecting to answer insulting questions.
• Tools manipulation. In essence, LLMs are trained as text
generators over massive plain text corpora, thus performing
less well on the tasks that are not best expressed in the
form of text (e.g., numerical computation). In addition, their
capacities are also limited to the pre-training data, e.g., the
inability to capture up-to-date information. To tackle these
issues, a recently proposed technique is to employ external
tools to compensate for the deficiencies of LLMs [80, 81].
For example, LLMs can utilize the calculator for accurate
computation [80] and employ search engines to retrieve
unknown information [81]. More recently, ChatGPT has
enabled the mechanism of using external plugins (existing
or newly created apps)12, which are by analogy with the
“eyes and ears” of LLMs. Such a mechanism can broadly
expand the scope of capacities for LLMs.
In addition, many other factors (e.g., the upgrade of
hardware) also contribute to the success of LLMs. Currently,
we limit our discussion to the major technical approaches
and key findings for developing LLMs.
2.2 Technical Evolution of GPT-series Models
Due to the excellent capacity in communicating with humans, ChatGPT has ignited the excitement of the AI community since its release. ChatGPT is developed based on the
powerful GPT model with specially optimized conversation
capacities. Considering the ever-growing interest in ChatGPT and GPT models, we add a special discussion about the
technical evolution of the GPT-series models, to briefly summarize the progress how they have been developed in the
past years. Meanwhile, we drew a schematic diagram depicting the technological evolution of the GPT-series models
in Figure 4. The basic principle underlying GPT models is
to compress the world knowledge into the decoder-only
Transformer model by language modeling, such that it can
recover (or memorize) the semantics of world knowledge
and serve as a general-purpose task solver. Two key points
to the success are (I) training decoder-only Transformer
language models that can accurately predict the next word
and (II) scaling up the size of language models. Overall, the
research of OpenAI on LLMs can be roughly divided into
the following stages13
.
Early Explorations. According to one interview with Ilya
Sutskever14 (a co-founder and chief scientist of OpenAI),
the idea of approaching intelligent systems with language
models was already explored in the early days of OpenAI, while it was attempted with recurrent neural networks (RNN) [121]. With the advent of Transformer, OpenAI
developed two initial GPT models, namely GPT-1 [122] and
GPT-2 [26], which can be considered as the foundation to
more powerful models subsequently i.e., GPT-3 and GPT-4.
• GPT-1. In 2017, the Transformer model [22] was introduced by Google, and the OpenAI team quickly adapted
their language modeling work to this new neural network
architecture. They released the first GPT model in 2018,
i.e., GPT-1 [122], and coined the abbreviation term GPT
as the model name, standing for Generative Pre-Training.
GPT-1 was developed based on a generative, decoder-only
Transformer architecture, and adopted a hybrid approach of
unsupervised pretraining and supervised fine-tuning. GPT1 has set up the core architecture for the GPT-series models
and established the underlying principle to model natural
language text, i.e., predicting the next word.
• GPT-2. Following a similar architecture of GPT-1,
GPT-2 [26] increased the parameter scale to 1.5B, which
was trained with a large webpage dataset WebText. As
claimed in the paper of GPT-2, it sought to perform
tasks via unsupervised language modeling, without explicit
fine-tuning using labeled data. To motivate the approach,
they introduced a probabilistic form for multi-task solving,
i.e., p(output|input, task) (similar approaches have been
adopted in [123]), which predicts the output conditioned on
the input and task information. To model this conditional
probability, language text can be naturally employed as a
unified way to format input, output and task information.
In this way, the process of solving a task can be cast as a
word prediction problem for generating the solution text.
Further, they introduced a more formal claim for this idea:
“Since the (task-specific) supervised objective is the same
as the unsupervised (language modeling) objective but only
evaluated on a subset of the sequence, the global minimum
of the unsupervised objective is also the global minimum
of the supervised objective (for various tasks)” [26]15. A
basic understanding of this claim is that each (NLP) task
can be considered as the word prediction problem based
on a subset of the world text. Thus, unsupervised language
modeling could be capable in solving various tasks, if it was
trained to have sufficient capacity in recovering the world
text. These early discussion in GPT-2’s paper echoed in the
interview of Ilya Sutskever by Jensen Huang: “What the
neural network learns is some representation of the process
that produced the text. This text is actually a projection of
the world...the more accurate you are in predicting the next
word, the higher the fidelity, the more resolution you get in
this process...”16
.
Capacity Leap. Although GPT-2 is intended to be an “unsupervised multitask learner”, it overall has an inferior
performance compared with supervised fine-tuning stateof-the-art methods. Because it has a relatively small model
size, it has been widely fine-tuned in downstream tasks,
especially the dialog tasks [124, 125]. Based on GPT-2, GPT-3

"""

In [948]:
"""
split chunks
"""

text = ori_text#.replace('\n', '')
matches = re.finditer(r'\n', text)
print("text length:", len(text))

max_chunk_length = 2048
result_chunks = []

left = 0
right = 0
match = -1
while(match is not None):
    try:
        match = next(matches)
        midx = match.start()
    except:
        match = None
        midx = len(text)
    
    if midx - left < max_chunk_length:
        right = midx
    else:
        # text split is longer than max_chuck_length
        if left >= right:
            while(midx - left >= max_chunk_length):
                chunk = text[left:left+max_chunk_length]
                left += max_chunk_length
                right += max_chunk_length
                result_chunks.append(chunk)
            right = midx
        chunk = text[left:right+1]
        left = right+1
        right = midx
        result_chunks.append(chunk)
        
if left < right: # get last split
    chunk = text[left:right+1]
    result_chunks.append(chunk)
    
print("num chunk:", len(result_chunks))

text length: 36226
num chunk: 19


In [949]:
%%time
"""
extract ToC
"""

api_server_url = "http://localhost:21122"
instruct = (
    "You are a Table of Contents extractor. "
    "User will speak to you questions. "
    "You must reply only with [목차(Table of Contents)] part extracted from the questions. "
    "You must keep original text. Do not change original text. "
    "And you must not involve [dotted line, page number, 제목(title), 참고문헌(reference), 부록(appendix), content, explanation, summary, predicted]. "
    "When there is no Table of Contents, you must reply with \"없음\". "
    "do not write explanations."
)
prompt_template = """### System:
{instruct}
### User: {data}

### Assistant:"""
def send_extract_toc(tocs):
    global idx
    # for _ in range(2):
    while(1):
        if idx > len_data - 1:
            break
        lock.acquire()
        pidx = idx
        data = result_chunks[pidx]
        idx += 1
        lock.release()
        
        print(f"{idx}/{len_data}", '\t\t\t\t\t\t', end='\r')#
        
        # response
        prompt = prompt_template.format(
            instruct=instruct,
            data=data
        )
        input_json = {
            "model_name": "MDIEM-toc3", #"OLLM-Small_2024.01.16", #"MDIEM-toc3",
            "prompt": prompt,
            "temperature": 0.7,
            "top_p": 0.8,
            "max_new_tokens": 512,
        }

        ret = requests.post(
            api_server_url + "/worker_generate",
            json=input_json,
            timeout=30,
        )

        result = ret.json()['text'][len(input_json['prompt'])+1:]
        tocs[pidx] = result

tocs = [""] * len(result_chunks)
threads = []
idx = 0
lock = threading.Lock()
len_data = len(result_chunks)
n_thread = 16

for i in range(n_thread):
    t = threading.Thread(target=send_extract_toc, args=(tocs,)) # 
    t.start()
    threads.append(t)
    
for t in threads:
    t.join()
    
    
first_toc = '\n'.join(tocs)
# response
prompt = prompt_template.format(
    instruct=instruct,
    data=first_toc
)
input_json = {
    "model_name": "MDIEM-toc3", #"OLLM-Small_2024.01.16", # 
    "prompt": prompt,
    "temperature": 0.7,
    "top_p": 0.8,
    "max_new_tokens": 1024,
}

ret = requests.post(
    api_server_url + "/worker_generate",
    json=input_json,
    timeout=30,
)

second_toc = ret.json()['text'][len(input_json['prompt'])+1:]
print(second_toc)

1 INTRODUCTION
2 OVERVIEW
2.1 Background for LLMs
2.2 Technical Evolution of GPT-series Models

CPU times: user 65.7 ms, sys: 23.6 ms, total: 89.3 ms
Wall time: 2.66 s


In [ ]:
print(first_toc)

In [951]:
"""
make ToC-Contents Groups
"""

toc_string = re.sub(r'\n+', '\n', second_toc)
parts = toc_string.split('\n')

MIN_SPLIT_LEN = 1

def remove_empty(parts):
    new_parts = []
    for part in parts:
        if len(part) <= MIN_SPLIT_LEN: continue
        new_parts.append(part)
    return new_parts

parts = remove_empty(parts)

parts_indexs = []
for part in parts:
    matched = [match.start() for match in re.finditer(part, ori_text)]
    for match in matched:
        parts_indexs.append((part, match))
    print(part, matched)
    
parts_indexs = sorted(parts_indexs, key=lambda x: x[1])
print(parts_indexs)

toc_dict = {}
for pdata1, pdata2 in zip(parts_indexs, parts_indexs[1:] + [('', len(ori_text)-1)]):
    name, left = pdata1
    left += len(name)
    _, right = pdata2
    
    if right - left > MIN_SPLIT_LEN:
        if name not in toc_dict:
            toc_dict[name] = [ori_text[left:right]]
        else:
            toc_dict[name].append(ori_text[left:right])

1 INTRODUCTION [0]
2 OVERVIEW [13829]
2.1 Background for LLMs [13978]
2.2 Technical Evolution of GPT-series Models [31719]
[('1 INTRODUCTION', 0), ('2 OVERVIEW', 13829), ('2.1 Background for LLMs', 13978), ('2.2 Technical Evolution of GPT-series Models', 31719)]


In [952]:
%%time
"""
Summary ToC-Contents Groups
"""

MIN_CONTENT_LEN = 10

api_server_url = "http://localhost:21122"
# I want you to act as a summaryist. I will give you sentences. "
# f"I want you to only reply with a summarization based on the sentences. do not write explanations.
# "do not write explanations. summarize the sentences as briefly as possible: {query}"
instruct = (
    "do not write explanations. 다음 문장을 한글로 번역하고 최대한 짧게 요약해주세요: {query}"
)
prompt_template = """### System:
This is a system prompt, please behave and help the user.

### User: {instruct}

### Assistant:"""
def send_summary(summaries):
    global idx
    # for _ in range(2):
    while(1):
        if idx > len_data - 1:
            break
        lock.acquire()
        pidx = idx
        data = toc_dict.get(parts[pidx])
        idx += 1
        lock.release()
        
        print(f"{idx}/{len_data}", '\t\t\t\t\t\t', end='\r')#
        
        if data is None:
            continue
        data = '\n'.join(data)
        if len(data) < MIN_CONTENT_LEN:
            # summaries[pidx] = data
            continue
        
        # response
        prompt = prompt_template.format(
            instruct=instruct.format(query=data),
        )
        input_json = {
            "model_name": "OLLM-Large_2023.11.20",# "MDIEM-toc3",
            "prompt": prompt,
            "temperature": 0.8,
            "top_p": 0.7,
            "max_new_tokens": 1024,
        }

        ret = requests.post(
            api_server_url + "/worker_generate",
            json=input_json,
            timeout=300,
        )

        result = ret.json()['text'][len(input_json['prompt'])+1:]
        summaries[pidx] = result

summaries = [""] * len(parts)
threads = []
idx = 0
lock = threading.Lock()
len_data = len(parts)
n_thread = 16

for i in range(n_thread):
    t = threading.Thread(target=send_summary, args=(summaries,)) # 
    t.start()
    threads.append(t)
    
for t in threads:
    t.join()

CPU times: user 21.7 ms, sys: 3.94 ms, total: 25.7 ms
Wall time: 11.4 s


In [953]:
for idx in range(len(parts)):
    print(parts[idx], '\n', summaries[idx], '\n')

1 INTRODUCTION 
 The given text is a summary of a research paper that discusses the development and applications of large language models (LLMs). The paper highlights the differences between LLMs and previous generations of language models, such as statistical language models (SLMs), neural language models (NLMs), and pre-trained language models (PLMs). It emphasizes the significant impact of LLMs on various domains, including natural language processing (NLP), information retrieval (IR), and computer vision (CV). The paper also discusses the challenges associated with LLMs, such as the lack of understanding of their emergent abilities, the difficulty in training them, and the need for aligning their output with human values. The paper concludes with a discussion of practical guidelines for prompt design and a review of LLM applications in different domains. 

2 OVERVIEW 
 이 섹션에서는 LLM의 배경에 대한 개요를 제공한 다음 GPT-시리즈 모델의 기술적 진화를 요약합니다. 

2.1 Background for LLMs 
  

2.2 Technical Evolution o

In [ ]:
for part in parts:
    if part in toc_dict:
        print(part, '\n', len(toc_dict[part]), '\n', toc_dict[part], '\n')

In [934]:
from transformers import AutoTokenizer
# model_path = "/workspaces/disk0/data/llm_weights/Platypus2-70B-instruct/"
# model_path = "/workspaces/disk0/data/llm_weights/vicuna-13b-v1.5/"
# model_path = "/data/llm_weights/gptq/MingAI-70B-chat-orca_v0.42_2_dpo-GPTQ/"
model_path = "/data/llm_weights/gptq/CodeLlama-70B-Instruct-GPTQ/"
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    model_max_length=16384,
    padding_side="right",
    use_fast=False,
)
tokenizer.pad_token = tokenizer.unk_token
from transformers.trainer_pt_utils import LabelSmoother
import torch
IGNORE_TOKEN_ID = LabelSmoother.ignore_index

In [933]:
input_ids = tokenizer("hi")['input_ids']
# input_ids
tokenizer.decode(input_ids)

'<s> hi'

In [935]:
input_ids = tokenizer("hi")['input_ids']
# input_ids
tokenizer.decode(input_ids)

'<s>hi'

In [958]:
1425115*100/2.37

60131434.59915612